In [0]:
%pip install polars

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/31/9e/21e05959323883abcee799837d8cac08adf10a48c233432993757e41791a/polars-1.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/36.0 MB ? eta -:--:--
   ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/36.0 MB 104.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/36.0 MB 224.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━ 19.3/36.0 MB 229.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━ 21.0/36.0 MB 222.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━ 21.0/36.0 MB 222.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 25.2/36.0 MB 83.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━ 31.8/36.0 MB 194.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 35.2/36.0 MB 161.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 36.0/36.0 

In [0]:
import dlt
import polars as pl
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import math
import gc

In [0]:

parquet_data = "/dbfs/mnt/registro-informacion-completa/parquet/Graduados.parquet"
# Cargar datos en un DataFrame de Polars
graduados = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/registro-informacion-completa/parquet/Info_Notas_HA_Actual.parquet"
# Cargar datos en un DataFrame de Polars
Info_Notas_HA_Actual = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/registro-informacion-completa/parquet/Historia_Academica.parquet"
# Cargar datos en un DataFrame de Polars
historia_acadmica = pl.read_parquet(parquet_data)

#Programas_Estudiante
parquet_data = "/dbfs/mnt/registro-informacion-completa/parquet/Programas_Estudiante.parquet"
# Cargar datos en un DataFrame de Polars
programas_estudiante = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/registro-informacion-completa/parquet/Oferta_Academica_Cursos.parquet"
# Cargar datos en un DataFrame de Polars
oferta_academica_cursos = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/registro-informacion-completa/parquet/Oferta_Academica_Franjas.parquet"
# Cargar datos en un DataFrame de Polars
oferta_academica_franjas = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/registro-informacion-completa/parquet/Docentes_Cursos.parquet"
# Cargar datos en un DataFrame de Polars
docentes_cursos = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/registro-informacion-completa/parquet/Personas.parquet"
# Cargar datos en un DataFrame de Polars
personas = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/datos-externos-observatorio/Banner/Info_Admitidos_General.parquet"
# Cargar datos en un DataFrame de Polars
info_admitidos_general = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/productos-datos-observatorio/estudios/matriculados/MATRICULAS_PAGO_BASE_BECAS.parquet"
# Cargar datos en un DataFrame de Polars
matriculas_pago_base_becas = pl.read_parquet(parquet_data)

parquet_data = "/dbfs/mnt/datos-externos-observatorio/parquet/retiros_201210_201920.parquet"
# Cargar datos en un DataFrame de Polars
retiros_201210_201920 = pl.read_parquet(parquet_data)

In [0]:
#dim_horario

dim_horario= oferta_academica_franjas.select(['FECHA_INICIO_FRANJA', 'FECHA_FIN_FRANJA', 'HORA_INICIO_FRANJA',
       'HORA_FIN_FRANJA', 'LUNES', 'MARTES', 'MIERCOLES', 'JUEVES',
       'VIERNES', 'SABADO', 'DOMINGO'])

dim_horario= dim_horario.unique(subset=['FECHA_INICIO_FRANJA', 'FECHA_FIN_FRANJA','HORA_INICIO_FRANJA',
       'HORA_FIN_FRANJA', 'LUNES', 'MARTES', 'MIERCOLES', 'JUEVES',
       'VIERNES', 'SABADO', 'DOMINGO'])

# Agregar un índice
dim_horario = dim_horario.with_row_index(name="ID_HORARIO")

columnas_seleccionadas= ['HORA_INICIO_FRANJA',
       'HORA_FIN_FRANJA', 'LUNES', 'MARTES', 'MIERCOLES', 'JUEVES',
       'VIERNES', 'SABADO', 'DOMINGO']

# Reemplazar valores null y cadenas vacías por "No especificado"
dim_horario = dim_horario.fill_null("No especificado")

dim_horario = dim_horario.with_columns([
    pl.when(pl.col(col) == "").then(pl.lit("No especificado")).otherwise(pl.col(col)).alias(col) 
    for col in columnas_seleccionadas
])



#dim_grupo_horario Y dim_union_grupo_horario
dim_union_grupo_horario= oferta_academica_franjas.select(['PERIODO','CRN','FECHA_INICIO_FRANJA', 'FECHA_FIN_FRANJA', 'HORA_INICIO_FRANJA',
       'HORA_FIN_FRANJA', 'LUNES', 'MARTES', 'MIERCOLES', 'JUEVES',
       'VIERNES', 'SABADO', 'DOMINGO'])

# Reemplazar valores null y cadenas vacías por "No especificado"
dim_union_grupo_horario = dim_union_grupo_horario.fill_null("No especificado")

dim_union_grupo_horario = dim_union_grupo_horario.with_columns([
    pl.when(pl.col(col) == "").then(pl.lit("No especificado")).otherwise(pl.col(col)).alias(col) 
    for col in columnas_seleccionadas
])

dim_union_grupo_horario= dim_union_grupo_horario.join(dim_horario,on=['FECHA_INICIO_FRANJA', 'FECHA_FIN_FRANJA', 'HORA_INICIO_FRANJA',
       'HORA_FIN_FRANJA', 'LUNES', 'MARTES', 'MIERCOLES', 'JUEVES',
       'VIERNES', 'SABADO', 'DOMINGO'],how="left")

dim_union_grupo_horario_pre_gr= dim_union_grupo_horario.select(['PERIODO','CRN','FECHA_INICIO_FRANJA', 'FECHA_FIN_FRANJA', 'HORA_INICIO_FRANJA',
       'HORA_FIN_FRANJA', 'LUNES', 'MARTES', 'MIERCOLES', 'JUEVES',
       'VIERNES', 'SABADO', 'DOMINGO',"ID_HORARIO"])


dim_union_grupo_horario= dim_union_grupo_horario_pre_gr.group_by(["PERIODO","CRN"]).agg([
    pl.col("ID_HORARIO").cast(pl.Utf8).str.concat("").alias("ID_GRUPO_HORARIO")]
)

dim_grupo_horario= dim_union_grupo_horario.unique("ID_GRUPO_HORARIO").select(["ID_GRUPO_HORARIO"])

dim_union_grupo_horario = dim_union_grupo_horario_pre_gr.join(dim_union_grupo_horario.select(['PERIODO','CRN',"ID_GRUPO_HORARIO"]),on=["PERIODO","CRN"],how="left")


In [0]:
# info_cursos_homologados ETL
# Filtrar filas donde 'ESTATUS_CURSO' sea igual a 'EXAMENES'
info_cursos_homologados = Info_Notas_HA_Actual.filter(pl.col("ESTATUS_CURSO") == "EXAMENES")

# Seleccionar columnas necesarias
info_cursos_homologados = info_cursos_homologados.select([
    "NOMBRE_CURSO_EXAMEN", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL", "NUMERO_CREDITOS", "ESTATUS_CURSO",
    "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTADO_MATERIA", "DESCRIPCION_ESTADO_MATERIA",
    "MATERIA", "CODIGO_MODO_DE_CALIFICACION", "DESCRIPCION_MODO_DE_CALIFICACION", "CODIGO_FACULTAD"
])

# Reordenar columnas
info_cursos_homologados = info_cursos_homologados.select([
    "MATERIA", "NOMBRE_CURSO_EXAMEN", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL", "NUMERO_CREDITOS",
    "ESTATUS_CURSO", "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTADO_MATERIA",
    "DESCRIPCION_ESTADO_MATERIA", "CODIGO_MODO_DE_CALIFICACION", "DESCRIPCION_MODO_DE_CALIFICACION",
    "CODIGO_FACULTAD"
])

# Quitar duplicados en base a las columnas 'MATERIA' y 'NOMBRE_CURSO_EXAMEN'
info_cursos_homologados = info_cursos_homologados.unique(subset=["MATERIA", "NOMBRE_CURSO_EXAMEN"])

# Reordenar columnas de nuevo
info_cursos_homologados = info_cursos_homologados.select([
    "MATERIA", "CODIGO_FACULTAD", "NOMBRE_CURSO_EXAMEN", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL",
    "NUMERO_CREDITOS", "ESTATUS_CURSO", "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO",
    "ESTADO_MATERIA", "DESCRIPCION_ESTADO_MATERIA", "CODIGO_MODO_DE_CALIFICACION", "DESCRIPCION_MODO_DE_CALIFICACION"
])

# Renombrar columnas
info_cursos_homologados = info_cursos_homologados.rename({
    "NOMBRE_CURSO_EXAMEN": "NOMBRE_CURSO_LARGO",
    "NUMERO_CREDITOS": "HORAS_CREDITO"
})

# Remover columnas innecesarias
info_cursos_homologados = info_cursos_homologados.drop([
    "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTADO_MATERIA", "DESCRIPCION_ESTADO_MATERIA",
    "CODIGO_MODO_DE_CALIFICACION", "DESCRIPCION_MODO_DE_CALIFICACION"
])

info_cursos_homologados = info_cursos_homologados.with_columns(pl.lit("0").cast(pl.String).alias("PERIODO"),
                                                              pl.lit("No especeficado").alias("CODIGO_ASIGNATURA"),
                                                              pl.lit("No especeficado").alias("NUMERO_CURSO"),
                                                              pl.lit("No especeficado").alias("DESCRIPCION_FACULTAD"),
                                                              pl.lit("No especeficado").alias("CODIGO_DEPARTAMENTO"),
                                                              pl.lit("No especeficado").alias("DESCRIPCION_DEPARTAMENTO"),
                                                              pl.lit("No especeficado").alias("DESCRIPCION_MODO_DE_CALIFICACION"),
                                                              pl.lit("No especeficado").alias("ATRIBUTO_SECCION"),
                                                              pl.lit("No especeficado").alias("ATRIBUTO_CURSO"),
                                                              pl.col("HORAS_CREDITO").cast(pl.String)
                                                              )

info_cursos_homologados= info_cursos_homologados.select(["PERIODO", "CODIGO_ASIGNATURA", "NUMERO_CURSO", "MATERIA", "CODIGO_FACULTAD", "DESCRIPCION_FACULTAD",
    "CODIGO_DEPARTAMENTO", "DESCRIPCION_DEPARTAMENTO", "HORAS_CREDITO", "NOMBRE_CURSO_LARGO",
    "DESCRIPCION_MODO_DE_CALIFICACION", "ATRIBUTO_SECCION", "ATRIBUTO_CURSO"])

In [0]:
# dim_curso

# Seleccionar columnas necesarias
dim_curso = oferta_academica_cursos.select([
    "PERIODO", "CODIGO_ASIGNATURA", "NUMERO_CURSO", "MATERIA", "CODIGO_FACULTAD", "DESCRIPCION_FACULTAD",
    "CODIGO_DEPARTAMENTO", "DESCRIPCION_DEPARTAMENTO", "HORAS_CREDITO", "NOMBRE_CURSO_LARGO",
    "DESCRIPCION_MODO_DE_CALIFICACION", "ATRIBUTO_SECCION", "ATRIBUTO_CURSO"
])

# Reemplazar valores nulos con "No especificado" en columnas específicas
dim_curso = dim_curso.with_columns([
    pl.col("ATRIBUTO_SECCION").fill_null("No especificado"),
    pl.col("ATRIBUTO_CURSO").fill_null("No especificado"),
    pl.col("MATERIA").fill_null("No especificado"),
    pl.col("CODIGO_FACULTAD").fill_null("No especificado"),
    pl.col("DESCRIPCION_FACULTAD").fill_null("No especificado"),
    pl.col("CODIGO_DEPARTAMENTO").fill_null("No especificado"),
    pl.col("DESCRIPCION_DEPARTAMENTO").fill_null("No especificado"),
    pl.col("HORAS_CREDITO").fill_null("No especificado"),
    pl.col("NOMBRE_CURSO_LARGO").fill_null("No especificado"),
    pl.col("DESCRIPCION_MODO_DE_CALIFICACION").fill_null("No especificado"),
    pl.col("CODIGO_ASIGNATURA").fill_null("No especificado"),
    pl.col("NUMERO_CURSO").fill_null("No especificado"),
])

# Filtrar filas con valores específicos
dim_curso = dim_curso.filter(pl.col("NOMBRE_CURSO_LARGO") != "No especificado")

# Limpieza de texto
dim_curso = dim_curso.with_columns([
    pl.col("MATERIA").str.strip_chars()
])

# Unir con otra tabla (info_cursos_homologados)
dim_curso = pl.concat([dim_curso, info_cursos_homologados])

dim_curso = dim_curso.sort("PERIODO",descending=True)

# Quitar duplicados en base a la columna "MATERIA"
dim_curso = dim_curso.unique(subset=["MATERIA"],keep="first")

# Agregar un índice
dim_curso = dim_curso.with_row_index(name="ID_MATERIA")

# Reordenar columnas y renombrar "Índice" a "ID_MATERIA"
column_order = ["ID_MATERIA", "MATERIA", "CODIGO_FACULTAD", "DESCRIPCION_FACULTAD", 
                "CODIGO_DEPARTAMENTO", "DESCRIPCION_DEPARTAMENTO", "HORAS_CREDITO", 
                "NOMBRE_CURSO_LARGO", "DESCRIPCION_MODO_DE_CALIFICACION", 
                "ATRIBUTO_SECCION", "ATRIBUTO_CURSO", "CODIGO_ASIGNATURA", "NUMERO_CURSO"]
                
dim_curso = dim_curso.select(column_order)

In [0]:
# info_programas_periodo

columnas_seleccionadas = ["PERIODO", "CODIGO_ESTUDIANTE", "CODIGO_NIVEL_PROGRAMA_1", "DESCRIPCION_NIVEL_PROGRAMA_1", 
    "CODIGO_FACULTAD_PROGRAMA_1", "DESCRIPCION_FACULTAD_PROGRAMA_1", "CODIGO_PROGRAMA_1", 
    "DESCRIPCION_PROGRAMA_1", "CODIGO_DEPARTAMENTO_PROGRAMA_1", "DESCRIPCION_DEPARTAMENTO_PROGRAMA_1", 
    "CODIGO_NIVEL_PROGRAMA_2", "DESCRIPCION_NIVEL_PROGRAMA_2", "CODIGO_FACULTAD_PROGRAMA_2", 
    "DESCRIPCION_FACULTAD_PROGRAMA_2", "CODIGO_PROGRAMA_2", "DESCRIPCION_PROGRAMA_2", 
    "CODIGO_DEPARTAMENTO_PROGRAMA_2", "DESCRIPCION_DEPARTAMENTO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_3", 
    "DESCRIPCION_NIVEL_PROGRAMA_3", "CODIGO_FACULTAD_PROGRAMA_3", "DESCRIPCION_FACULTAD_PROGRAMA_3", 
    "CODIGO_PROGRAMA_3", "DESCRIPCION_PROGRAMA_3", "CODIGO_DEPARTAMENTO_PROGRAMA_3", 
    "DESCRIPCION_DEPARTAMENTO_PROGRAMA_3"]

# Seleccionar columnas necesarias
info_programas_periodo = programas_estudiante.select(columnas_seleccionadas)

# Reemplazar valores null y cadenas vacías por "No especificado"
info_programas_periodo = info_programas_periodo.fill_null("No especificado")

info_programas_periodo = info_programas_periodo.with_columns([
    pl.when(pl.col(col) == "").then(pl.lit("No especificado")).otherwise(pl.col(col)).alias(col) 
    for col in columnas_seleccionadas
])


info_programas_periodo = info_programas_periodo.with_columns(
    [pl.col(col).fill_null("No especificado").alias(col) for col in columnas_seleccionadas]
)

# Mostrar el resultado final

In [0]:
# dim_programa

# Seleccionar las columnas necesarias
columnas_seleccionadas = [
    "CODIGO_NIVEL_PROGRAMA_1", "DESCRIPCION_NIVEL_PROGRAMA_1", "CODIGO_FACULTAD_PROGRAMA_1", "DESCRIPCION_FACULTAD_PROGRAMA_1", 
    "CODIGO_PROGRAMA_1", "DESCRIPCION_PROGRAMA_1", "CODIGO_DEPARTAMENTO_PROGRAMA_1", "DESCRIPCION_DEPARTAMENTO_PROGRAMA_1",
    "CODIGO_NIVEL_PROGRAMA_2", "DESCRIPCION_NIVEL_PROGRAMA_2", "CODIGO_FACULTAD_PROGRAMA_2", "DESCRIPCION_FACULTAD_PROGRAMA_2",
    "CODIGO_PROGRAMA_2", "DESCRIPCION_PROGRAMA_2", "CODIGO_DEPARTAMENTO_PROGRAMA_2", "DESCRIPCION_DEPARTAMENTO_PROGRAMA_2",
    "CODIGO_NIVEL_PROGRAMA_3", "DESCRIPCION_NIVEL_PROGRAMA_3", "CODIGO_FACULTAD_PROGRAMA_3", "DESCRIPCION_FACULTAD_PROGRAMA_3",
    "CODIGO_PROGRAMA_3", "DESCRIPCION_PROGRAMA_3", "CODIGO_DEPARTAMENTO_PROGRAMA_3", "DESCRIPCION_DEPARTAMENTO_PROGRAMA_3"
]
dim_programa = programas_estudiante.select(columnas_seleccionadas)

# Remover duplicados en las columnas específicas
dim_programa = dim_programa.unique(
    subset=["CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2", "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"]
)

# Agregar columna de índice como ID_PROGRAMA
dim_programa = dim_programa.with_row_index(name="ID_PROGRAMA")
#Cambiar de tipo de columna a str
dim_programa = dim_programa.with_columns([pl.col("ID_PROGRAMA").cast(str).alias("ID_PROGRAMA")])

# Reemplazar valores nulos y valores vacíos con "No especificado"
dim_programa = dim_programa.with_columns([pl.col(col).str.strip_chars() for col in columnas_seleccionadas])

dim_programa = dim_programa.with_columns([pl.col(col).fill_null("No especificado").alias(col) for col in columnas_seleccionadas])

dim_programa = dim_programa.with_columns([
    pl.when(pl.col(col) == "").then(pl.lit("No especificado")).otherwise(pl.col(col)).alias(col) 
    for col in columnas_seleccionadas
])

# Reordenar las columnas (si es necesario, esto depende de tu necesidad específica)
orden_columnas = ["ID_PROGRAMA"] + columnas_seleccionadas
dim_programa = dim_programa.select(orden_columnas)

In [0]:
# info_estudiantes_programa_periodo

# Seleccionar columnas necesarias
info_estudiantes_programa_periodo = Info_Notas_HA_Actual.select([
    "PERIODO", "CODIGO_ESTUDIANTE", "SECCION", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL",
    "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTADO_MATERIA",
    "DESCRIPCION_ESTADO_MATERIA", "MATERIA", "CODIGO_DOCENTE", "CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"
])

# Simular el join con el dataframe 'dim_curso'
# Asegúrate de cargar o definir 'dim_curso' de manera similar a este dataframe
info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.join(dim_curso.select(["ID_MATERIA", "HORAS_CREDITO","MATERIA"]), on="MATERIA", how="left")

info_estudiantes_programa_periodo=info_estudiantes_programa_periodo.with_columns([pl.col("ID_MATERIA").alias("dim_curso.ID_MATERIA"),
                                                pl.col("HORAS_CREDITO").alias("dim_curso.HORAS_CREDITO")])

# Ordenar las filas por 'PERIODO' en orden descendente
info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.sort("PERIODO")

# Seleccionar columnas específicas después del join
info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.select([
    "PERIODO", "CODIGO_ESTUDIANTE", "SECCION", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL",
    "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTADO_MATERIA", 
    "DESCRIPCION_ESTADO_MATERIA", "CODIGO_DOCENTE", "CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3", "dim_curso.ID_MATERIA", "dim_curso.HORAS_CREDITO"
])

# Reemplazar valores nulos y valores vacíos con "No especificado"
info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.with_columns([pl.col(col).str.strip_chars() for col in [
"CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"
]])

info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.with_columns([pl.col(col).fill_null("No especificado").alias(col) for col in [
"CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"
]])

info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.with_columns([
    pl.when(pl.col(col) == "").then(pl.lit("No especificado")).otherwise(pl.col(col)).alias(col) 
    for col in [
"CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"
]
])

# Simular el join con el dataframe 'dim_programa'
# Asegúrate de cargar o definir 'dim_programa' de manera similar a este dataframe
info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.join(dim_programa.select(["ID_PROGRAMA","CODIGO_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_PROGRAMA_3"]), on=["CODIGO_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_PROGRAMA_3"], how="left")

info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.with_columns([pl.col("ID_PROGRAMA").alias("dim_programa.ID_PROGRAMA")])

# Seleccionar columnas después del segundo join
info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.select(["PERIODO", "CODIGO_ESTUDIANTE", "dim_programa.ID_PROGRAMA"])

# Eliminar duplicados
info_estudiantes_programa_periodo = info_estudiantes_programa_periodo.unique(subset=["PERIODO", "CODIGO_ESTUDIANTE"])

In [0]:
# info_retiros_201920_hacia_atras
# Renombrar la columna 'CODIGO_EST' a 'CODIGO_ESTUDIANTE'
info_retiros_201920_hacia_atras = retiros_201210_201920.rename({"CODIGO_EST": "CODIGO_ESTUDIANTE"})

# Seleccionar columnas específicas
info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.select(["PERIODO", "CODIGO_ESTUDIANTE", "MATERIA", "NOMBRE_LARGO_CURSO", "CALIFICACION_FINAL","NRC"])

# Duplicar la columna 'CALIFICACION_FINAL' y renombrar la copia como 'DESCRIPCION_ESTADO_CURSO'
info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.with_columns(
    pl.col("CALIFICACION_FINAL").alias("DESCRIPCION_ESTADO_CURSO")
)

# Convertir 'DESCRIPCION_ESTADO_CURSO' a tipo texto
info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.with_columns(pl.col("DESCRIPCION_ESTADO_CURSO").cast(pl.Utf8))

# Reemplazar valores nulos en 'DESCRIPCION_ESTADO_CURSO' con "RETIRADO"
info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.with_columns(
    pl.col("DESCRIPCION_ESTADO_CURSO").fill_null("RETIRADO")
)

# Reemplazar valores nulos en otras columnas con "No especificado"
info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.with_columns([
    pl.col("PERIODO").fill_null("No especificado"),
    pl.col("CODIGO_ESTUDIANTE").fill_null("No especificado"),
    pl.col("MATERIA").fill_null("No especificado"),
    pl.col("NOMBRE_LARGO_CURSO").fill_null("No especificado"),
    pl.col("CALIFICACION_FINAL").fill_null("No especificado"),
    pl.col("DESCRIPCION_ESTADO_CURSO").fill_null("No especificado")
])

# Realizar el join con 'dim_curso' en la columna 'MATERIA'
# Cargar 'dim_curso' como un DataFrame de Polars con columnas 'MATERIA', 'ID_MATERIA', 'HORAS_CREDITO'


info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.join(dim_curso, on="MATERIA", how="left").select([
    "PERIODO", "CODIGO_ESTUDIANTE", "CALIFICACION_FINAL", "DESCRIPCION_ESTADO_CURSO", "ID_MATERIA", "HORAS_CREDITO","MATERIA","NRC"
])

# Renombrar 'DESCRIPCION_ESTADO_CURSO' a 'DESCRIPCION_ESTADO_MATERIA'
info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.rename({"DESCRIPCION_ESTADO_CURSO": "DESCRIPCION_ESTADO_MATERIA","MATERIA":"dim_curso.MATERIA"})

# Convertir columnas 'PERIODO' y 'CODIGO_ESTUDIANTE' a tipo texto
info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.with_columns([
    pl.col("PERIODO").cast(pl.Utf8),
    pl.col("CODIGO_ESTUDIANTE").cast(pl.Utf8)
])

# Realizar el join con 'info_estudiantes_programa_periodo' en columnas 'PERIODO' y 'CODIGO_ESTUDIANTE'
# Asegúrate de cargar 'info_estudiantes_programa_periodo' como un DataFrame de Polars

info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.join(info_estudiantes_programa_periodo, on=["PERIODO", "CODIGO_ESTUDIANTE"], how="left").select([
    "PERIODO", "CODIGO_ESTUDIANTE", "CALIFICACION_FINAL", "DESCRIPCION_ESTADO_MATERIA", "ID_MATERIA", "HORAS_CREDITO", "dim_programa.ID_PROGRAMA","NRC"
])

# Renombrar 'dim_programa.ID_PROGRAMA' a 'ID_PROGRAMA'
# info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.rename({"dim_programa.ID_PROGRAMA": "ID_PROGRAMA"})

info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.with_columns([pl.col("ID_MATERIA").alias("dim_curso.ID_MATERIA")])
info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.with_columns([pl.col("HORAS_CREDITO").alias("dim_curso.HORAS_CREDITO")])

columnas_a_crear=[
 'SECCION',
 'CALIFICACION_PARCIAL',
 'ESTATUS_CURSO',
 'CALIFICACION_PARCIAL_CONTEO',
 'CALIFICACION_FINAL_CONTEO',
 'ESTADO_MATERIA',
 'CODIGO_DOCENTE',
 'CODIGO_NIVEL_PROGRAMA_1',
 'CODIGO_PROGRAMA_1',
 'CODIGO_NIVEL_PROGRAMA_2',
 'CODIGO_PROGRAMA_2',
 'CODIGO_NIVEL_PROGRAMA_3',
 'CODIGO_PROGRAMA_3',
 "dim_curso.MATERIA"]

info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.with_columns([pl.lit(None).alias(col_name) for col_name in columnas_a_crear])

info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.rename({
    "NRC": "CRN"
})

info_retiros_201920_hacia_atras=info_retiros_201920_hacia_atras.with_columns(pl.col("CRN").cast(pl.Utf8))



info_retiros_201920_hacia_atras = info_retiros_201920_hacia_atras.join(dim_union_grupo_horario.select(["PERIODO","CRN","ID_GRUPO_HORARIO"]), on=["PERIODO","CRN"], how="left")

 

In [0]:
# info_profesor

# Seleccionar columnas necesarias
info_profesor = docentes_cursos.select([
    "CODIGO_DOCENTE", "PERIODO", "DOCENTE_PRINCIPAL", "SECCION", "PORCENTAJE_RESPONSABILIDAD", 
    "PORCENTAJE_SESION", "HORAS_SEMANAL", "INDICADOR_DOCENTE", "INDICADOR_ASESOR", "MATERIA"
])

# Mostrar el resultado final

In [0]:
# info_persona

# Seleccionar las columnas especificadas
info_persona = personas.select([
    "PIDM", "CODIGO_ESTUDIANTE", "PRIMER_NOMBRE", "SEGUNDO_NOMBRE", "NOMBRES", "PRIMER_APELLIDO",
    "SEGUNDO_APELLIDO", "APELLIDOS", "PRIMER_NOMBRE_IDENTITARIO", "SEGUNDO_NOMBRE_IDENTITARIO", 
    "PRIMER_APELLIDO_IDENTITARIO", "SEGUNDO_APELLIDO_IDENTITARIO", "CODIGO_TIPO_IDENTIFICACION", 
    "TIPO_IDENTIFICACION", "IDENTIFICACION", "TIPO_GENERO", "GENERO", "CODIGO_TIPO_NACIONALIDAD", 
    "NACIONALIDAD", "FECHA_DE_NACIMIENTO", "EDAD", "CODIGO_TIPO_ESTADO_CIVIL", "TIPO_ESTADO_CIVIL", 
    "FALLECIDO", "FECHA_DE_FALLECIMIENTO", "LOGIN", "CORREO_INSTITUCIONAL", "CORREO_PERSONAL", 
    "TELEFONO_RESIDENCIA", "TELEFONO_CELULAR", "TELEFONO_TRABAJO", "DIRECCION_RESIDENCIA", 
    "CODIGO_CIUDAD_RESIDENCIA", "CIUDAD_RESIDENCIA", "CODIGO_DEPARTAMENTO_RESIDENCIA", 
    "DEPARTAMENTO_RESIDENCIA", "CODIGO_PAIS_RESIDENCIA", "PAIS_RESIDENCIA", "DIRECCION_TRABAJO", 
    "CODIGO_CIUDAD_TRABAJO", "CIUDAD_TRABAJO", "CODIGO_DEPARTAMENTO_TRABAJO", "DEPARTAMENTO_TRABAJO", 
    "CODIGO_PAIS_TRABAJO", "PAIS_TRABAJO", "NOMBRE_PADRE", "TELEFONO_PADRE", "NOMBRE_MADRE", 
    "TELEFONO_MADRE", "REPRESENTANTE_FINANCIERO", "DIRECCION_REPRESENTANTE_FINANCIERO", 
    "TELEFONO_REPRESENTANTE_FINANCIERO", "CORREO_REPRESENTANTE_FINANCIERO", "INDICADOR_CONFIDENCIALIDAD", 
    "CIUDAD_RESIDENCIA_UG", "DEPARTAMENTO_RESIDENCIA_UG", "PAIS_RESIDENCIA_UG", "CIUDAD_TRABAJO_UG", 
    "DEPARTAMENTO_TRABAJO_UG", "PAIS_TRABAJO_UG", "CIUDAD_NACIMIENTO", "ESTRATO", "FACULTADES", 
    "CODIGO_FACULTAD"
])

# Mostrar el resultado final

In [0]:
# dim_estudiantes

# Filtrar por "DESCRIPCION_NIVEL_PROGRAMA" para obtener solo "PREGRADO"
dim_estudiantes = info_admitidos_general.filter(pl.col("DESCRIPCION_NIVEL_PROGRAMA") == "PREGRADO")

# Seleccionar las columnas deseadas
dim_estudiantes = dim_estudiantes.select([
    "PERIODO", "PERIODO_EFF", "DECISION_ADMISION", "DESCRIPCION_DECISION_ADMISION", "CODIGO_ESTUDIANTE", "APELLIDOS", "NOMBRES", "TIPO_ESTUDIANTE", "DESCRIPCION_TIPO_ESTUDIANTE", "TIPO_ESTUDIANTE_STDN", "DESCRIPCION_TIPO_ESTUDIANTE_STDN", "TIPO_ADMISION", "DESCRIPCION_TIPO_ADMISION", "ESTADO_ACADEMICO_INICIO_SEMESTRE", "FECHA_NACIMIENTO", "CIUDAD_NACIMIENTO", "NACIONALIDAD", "DESCRIPCION_NACIONALIDAD", "EDAD", "SEXO", "DESCRIPCION_SEXO", "ESTADO_CIVIL", "DESCRIPCION_ESTADO_CIVIL", "COHORTE", "COHORTE_DESCRIPCION", "DIRECCION", "CIUDAD", "CODIGO_CIUDAD", "CODIGO_DEPARTAMENTO", "CODIGO_PAIS", "INDICATIVO", "TELEFONO", "EXTENSION", "CELULAR", "CORREO_PERSONAL", "CORREO_UNIANDES", "ALERTAS", "ESTADO_ENTREGA_DOCUMENTOS_PG", "OBSERVACIONES_DOCUMENTOS_ENTREGADOS_PG", "ESTRATO", "CODIGO_COLEGIO", "CODIGO_DANE", "NOMBRE_COLEGIO", "CIUDAD_COLEGIO", "DEPARTAMENTO_COLEGIO", "TIPO_COLEGIO", "FECHA_GRADO_PG", "PEN_CURSO", "PEN_POSC", "PEN_TAM", "PEN_PGA", "UL_CURSO", "UL_POSC", "UL_TAM", "UL_PGA", "SE_HA_PRESENTADO", "SE_HA_MATRICULADO", "SOLICITA_APOYO_FINANCIERO", "TIPO_EXAMEN", "SNP", "PUNTAJE", "PLC", "PMA", "PSC", "PCN", "PIN", "EXAMEN_INTERNO", "EXAMEN_INTERNO_NOTA", "EXAMEN_HOMOLOGABLE", "PREGRADO_QS", "PREGRADO_QS_UNI", "PREGRADO_QS_VERFICADO", "APELLIDOS_PADRE", "NOMBRES_PADRE", "DIRECCION_PADRE", "CIUDAD_PADRE", "INDICATIVO_PADRE", "TELEFONO_PADRE", "EXTENSION_TELEFONO_PADRE", "CORREO_PADRE", "APELLIDOS_MADRE", "NOMBRES_MADRE", "DIRECCION_MADRE", "CIUDAD_MADRE", "INDICATIVO_MADRE", "TELEFONO_MADRE", "EXTENSION_TELEFONO_MADRE", "CORREO_MADRE", "APELLIDOS_ACUDIENTE", "NOMBRES_ACUDIENTE", "DIRECCION_ACUDIENTE", "CIUDAD_ACUDIENTE", "INDICATIVO_ACUDIENTE", "TELEFONO_ACUDIENTE", "EXTENSION_TELEFONO_ACUDIENTE", "CORREO_ACUDIENTE", "NUMERO_HERMANOS", "POSICION_ENTRE_HERMANOS", "ESTADO_CIVIL_PADRES", "NIVEL_EDUCATIVO_MADRE", "OCUPACION_MADRE", "NIVEL_EDUCATIVO_PADRE", "OCUPACION_PADRE", "TIPO_VIVIENDA", "CURSO_BACH_UN_COLEGIO", "NUMERO_COLEGIOS_BACH", "PERTENECE_MINORIA_ETNICA", "SISBEN", "PUN_SISBEN", "CUMPLE_I1", "CUMPLE_EI", "CUMPLE_RANK", "CUMPLE_PGA", "CUMPLE_UNAL", "CUMPLE_QS", "FORMAS_ADMISION", "FECHA_DOCUMENTOS", "INDICADOR_GRADUADO", "ESTADO_ENTREGA_DOCUMENTOS_PS", "OBERVACIONES_DOCUMENTOS_ENTREGADOS_PS", "NOMBRE_EMPRESA_ESTUDIANTE", "NOMBRE_SECTOR_EMPRESA_ESTUDIANTE", "CARGO_ESTUDIANTE", "DIRECCION_EMPRESA", "CIUDAD_EMPRESA", "PAIS_EMPRESA", "INDICATIVO_EMPRESA", "TELEFONO_EMPRESA", "EXTENSION_TELEFONO_EMPRESA", "UNIVERSIDAD_PREGRADO", "CIUDAD_UNIVERSIDAD", "PAIS_UNIVERSIDAD", "FECHA_INICIO_PREGRADO", "FECHA_FIN_PREGRADO", "TITULO_OBTENIDO", "PROMEDIO_ACUMULADO", "FECHA_GRADO_PS", "AREA_INVESTIGACION", "INTERES_ACADEMICO", "EXPERIENCIA_INVESTIGACION", "EXPERIENCIA_LABORAL", "DISTINCIONES", "REFERENCIAS_ACADEMICAS", "CONOCIMIENTOS_DEL_PROGRAMA", "FORMA_DE_FINANCIACION", "REQUIERE_APOYO_FINANCIERO", "LENGUA_MATERNA", "NIVEL_INGLES_LECTURA", "NIVEL_INGLES_ESCRITO", "NIVEL_INGLES_ORAL", "PORCENTAJE_TIEMPO_PARA_ESTUDIO", "ENTREGO_HOJA_VIDA", "ENTREGO_ACTA", "ENTREGO_NOTAS_PREGRADO", "MATRICULADO_BANNER", "MATRICULADO_FINANCIERA", "TIPO_FINANCIERA", "TOTAL_PAGO_FINANCIERA", "MANEJO_DATOS_SENSIBLES", "CIUDAD_RESIDENCIA_UG", "DEPARTAMENTO_RESIDENCIA_UG", "PAIS_RESIDENCIA_UG", "EDAD_ADM", "ESTADO_ACADEMICO_COMBINADO", "INSCRITO"
])

# Ordenar por "PERIODO" en orden descendente y quitar duplicados por "CODIGO_ESTUDIANTE"
dim_estudiantes = dim_estudiantes.sort("PERIODO",descending=False).unique(subset="CODIGO_ESTUDIANTE",keep="first")

# Reemplazar valores nulos con "No especificado" en las columnas seleccionadas
dim_estudiantes = dim_estudiantes.with_columns([
    pl.col(col_name).fill_null("No especificado") for col_name in [
        "TIPO_ESTUDIANTE_STDN", "DESCRIPCION_TIPO_ESTUDIANTE_STDN", "TIPO_ADMISION", "DESCRIPCION_TIPO_ADMISION", "ESTADO_ACADEMICO_INICIO_SEMESTRE", "CIUDAD_NACIMIENTO", "NACIONALIDAD", "DESCRIPCION_NACIONALIDAD", "EDAD", "SEXO", "DESCRIPCION_SEXO", "ESTADO_CIVIL", "DESCRIPCION_ESTADO_CIVIL", "COHORTE", "COHORTE_DESCRIPCION", "DIRECCION", "CIUDAD", "CODIGO_CIUDAD", "CODIGO_DEPARTAMENTO", "CODIGO_PAIS", "INDICATIVO", "TELEFONO", "EXTENSION", "CELULAR", "CORREO_PERSONAL", "CORREO_UNIANDES", "ALERTAS", "ESTADO_ENTREGA_DOCUMENTOS_PG", "OBSERVACIONES_DOCUMENTOS_ENTREGADOS_PG", "ESTRATO", "CODIGO_COLEGIO", "CODIGO_DANE", "NOMBRE_COLEGIO", "CIUDAD_COLEGIO", "DEPARTAMENTO_COLEGIO", "TIPO_COLEGIO", "PEN_CURSO", "PEN_POSC", "PEN_TAM", "PEN_PGA", "UL_CURSO", "UL_POSC", "UL_TAM", "UL_PGA", "SE_HA_PRESENTADO", "SE_HA_MATRICULADO", "SOLICITA_APOYO_FINANCIERO", "TIPO_EXAMEN", "SNP", "PUNTAJE", "PLC", "PMA", "PSC", "PCN", "PIN", "EXAMEN_INTERNO", "EXAMEN_INTERNO_NOTA", "EXAMEN_HOMOLOGABLE", "PREGRADO_QS", "PREGRADO_QS_UNI", "PREGRADO_QS_VERFICADO", "APELLIDOS_PADRE", "NOMBRES_PADRE", "DIRECCION_PADRE", "CIUDAD_PADRE", "INDICATIVO_PADRE", "TELEFONO_PADRE", "EXTENSION_TELEFONO_PADRE", "CORREO_PADRE", "APELLIDOS_MADRE", "NOMBRES_MADRE", "DIRECCION_MADRE", "CIUDAD_MADRE", "INDICATIVO_MADRE", "TELEFONO_MADRE", "EXTENSION_TELEFONO_MADRE", "CORREO_MADRE", "APELLIDOS_ACUDIENTE", "NOMBRES_ACUDIENTE", "DIRECCION_ACUDIENTE", "CIUDAD_ACUDIENTE", "INDICATIVO_ACUDIENTE", "TELEFONO_ACUDIENTE", "EXTENSION_TELEFONO_ACUDIENTE", "CORREO_ACUDIENTE", "NUMERO_HERMANOS", "POSICION_ENTRE_HERMANOS", "ESTADO_CIVIL_PADRES", "NIVEL_EDUCATIVO_MADRE", "OCUPACION_MADRE", "NIVEL_EDUCATIVO_PADRE", "OCUPACION_PADRE", "TIPO_VIVIENDA", "CURSO_BACH_UN_COLEGIO", "NUMERO_COLEGIOS_BACH", "PERTENECE_MINORIA_ETNICA", "SISBEN", "PUN_SISBEN", "CUMPLE_I1", "CUMPLE_EI", "CUMPLE_RANK", "CUMPLE_PGA", "CUMPLE_UNAL", "CUMPLE_QS", "FORMAS_ADMISION", "INDICADOR_GRADUADO", "PROMEDIO_ACUMULADO", "CONOCIMIENTOS_DEL_PROGRAMA", "FORMA_DE_FINANCIACION", "REQUIERE_APOYO_FINANCIERO", "LENGUA_MATERNA", "NIVEL_INGLES_LECTURA", "NIVEL_INGLES_ESCRITO", "NIVEL_INGLES_ORAL", "PORCENTAJE_TIEMPO_PARA_ESTUDIO", "MATRICULADO_BANNER", "MATRICULADO_FINANCIERA", "TIPO_FINANCIERA", "TOTAL_PAGO_FINANCIERA", "MANEJO_DATOS_SENSIBLES", "EDAD_ADM", "FECHA_GRADO_PS", "ESTADO_ACADEMICO_COMBINADO", "INSCRITO"
    ]
])

# Resultado final con valores nulos reemplazados

In [0]:
# dim_estudiantes_magister

# Filtrar por "DESCRIPCION_NIVEL_PROGRAMA" para obtener solo "PREGRADO"
dim_estudiantes_magister = info_admitidos_general.filter(pl.col("DESCRIPCION_NIVEL_PROGRAMA") == "MAGISTER")

# Seleccionar las columnas deseadas
dim_estudiantes_magister = dim_estudiantes_magister.select([
    "PERIODO", "PERIODO_EFF", "DECISION_ADMISION", "DESCRIPCION_DECISION_ADMISION", "CODIGO_ESTUDIANTE", "APELLIDOS", "NOMBRES", "TIPO_ESTUDIANTE", "DESCRIPCION_TIPO_ESTUDIANTE", "TIPO_ESTUDIANTE_STDN", "DESCRIPCION_TIPO_ESTUDIANTE_STDN", "TIPO_ADMISION", "DESCRIPCION_TIPO_ADMISION", "ESTADO_ACADEMICO_INICIO_SEMESTRE", "FECHA_NACIMIENTO", "CIUDAD_NACIMIENTO", "NACIONALIDAD", "DESCRIPCION_NACIONALIDAD", "EDAD", "SEXO", "DESCRIPCION_SEXO", "ESTADO_CIVIL", "DESCRIPCION_ESTADO_CIVIL", "COHORTE", "COHORTE_DESCRIPCION", "DIRECCION", "CIUDAD", "CODIGO_CIUDAD", "CODIGO_DEPARTAMENTO", "CODIGO_PAIS", "INDICATIVO", "TELEFONO", "EXTENSION", "CELULAR", "CORREO_PERSONAL", "CORREO_UNIANDES", "ALERTAS", "ESTADO_ENTREGA_DOCUMENTOS_PG", "OBSERVACIONES_DOCUMENTOS_ENTREGADOS_PG", "ESTRATO", "CODIGO_COLEGIO", "CODIGO_DANE", "NOMBRE_COLEGIO", "CIUDAD_COLEGIO", "DEPARTAMENTO_COLEGIO", "TIPO_COLEGIO", "FECHA_GRADO_PG", "PEN_CURSO", "PEN_POSC", "PEN_TAM", "PEN_PGA", "UL_CURSO", "UL_POSC", "UL_TAM", "UL_PGA", "SE_HA_PRESENTADO", "SE_HA_MATRICULADO", "SOLICITA_APOYO_FINANCIERO", "TIPO_EXAMEN", "SNP", "PUNTAJE", "PLC", "PMA", "PSC", "PCN", "PIN", "EXAMEN_INTERNO", "EXAMEN_INTERNO_NOTA", "EXAMEN_HOMOLOGABLE", "PREGRADO_QS", "PREGRADO_QS_UNI", "PREGRADO_QS_VERFICADO", "APELLIDOS_PADRE", "NOMBRES_PADRE", "DIRECCION_PADRE", "CIUDAD_PADRE", "INDICATIVO_PADRE", "TELEFONO_PADRE", "EXTENSION_TELEFONO_PADRE", "CORREO_PADRE", "APELLIDOS_MADRE", "NOMBRES_MADRE", "DIRECCION_MADRE", "CIUDAD_MADRE", "INDICATIVO_MADRE", "TELEFONO_MADRE", "EXTENSION_TELEFONO_MADRE", "CORREO_MADRE", "APELLIDOS_ACUDIENTE", "NOMBRES_ACUDIENTE", "DIRECCION_ACUDIENTE", "CIUDAD_ACUDIENTE", "INDICATIVO_ACUDIENTE", "TELEFONO_ACUDIENTE", "EXTENSION_TELEFONO_ACUDIENTE", "CORREO_ACUDIENTE", "NUMERO_HERMANOS", "POSICION_ENTRE_HERMANOS", "ESTADO_CIVIL_PADRES", "NIVEL_EDUCATIVO_MADRE", "OCUPACION_MADRE", "NIVEL_EDUCATIVO_PADRE", "OCUPACION_PADRE", "TIPO_VIVIENDA", "CURSO_BACH_UN_COLEGIO", "NUMERO_COLEGIOS_BACH", "PERTENECE_MINORIA_ETNICA", "SISBEN", "PUN_SISBEN", "CUMPLE_I1", "CUMPLE_EI", "CUMPLE_RANK", "CUMPLE_PGA", "CUMPLE_UNAL", "CUMPLE_QS", "FORMAS_ADMISION", "FECHA_DOCUMENTOS", "INDICADOR_GRADUADO", "ESTADO_ENTREGA_DOCUMENTOS_PS", "OBERVACIONES_DOCUMENTOS_ENTREGADOS_PS", "NOMBRE_EMPRESA_ESTUDIANTE", "NOMBRE_SECTOR_EMPRESA_ESTUDIANTE", "CARGO_ESTUDIANTE", "DIRECCION_EMPRESA", "CIUDAD_EMPRESA", "PAIS_EMPRESA", "INDICATIVO_EMPRESA", "TELEFONO_EMPRESA", "EXTENSION_TELEFONO_EMPRESA", "UNIVERSIDAD_PREGRADO", "CIUDAD_UNIVERSIDAD", "PAIS_UNIVERSIDAD", "FECHA_INICIO_PREGRADO", "FECHA_FIN_PREGRADO", "TITULO_OBTENIDO", "PROMEDIO_ACUMULADO", "FECHA_GRADO_PS", "AREA_INVESTIGACION", "INTERES_ACADEMICO", "EXPERIENCIA_INVESTIGACION", "EXPERIENCIA_LABORAL", "DISTINCIONES", "REFERENCIAS_ACADEMICAS", "CONOCIMIENTOS_DEL_PROGRAMA", "FORMA_DE_FINANCIACION", "REQUIERE_APOYO_FINANCIERO", "LENGUA_MATERNA", "NIVEL_INGLES_LECTURA", "NIVEL_INGLES_ESCRITO", "NIVEL_INGLES_ORAL", "PORCENTAJE_TIEMPO_PARA_ESTUDIO", "ENTREGO_HOJA_VIDA", "ENTREGO_ACTA", "ENTREGO_NOTAS_PREGRADO", "MATRICULADO_BANNER", "MATRICULADO_FINANCIERA", "TIPO_FINANCIERA", "TOTAL_PAGO_FINANCIERA", "MANEJO_DATOS_SENSIBLES", "CIUDAD_RESIDENCIA_UG", "DEPARTAMENTO_RESIDENCIA_UG", "PAIS_RESIDENCIA_UG", "EDAD_ADM", "ESTADO_ACADEMICO_COMBINADO", "INSCRITO"
])

# Ordenar por "PERIODO" en orden descendente y quitar duplicados por "CODIGO_ESTUDIANTE"
dim_estudiantes_magister = dim_estudiantes_magister.sort("PERIODO",descending=False).unique(subset="CODIGO_ESTUDIANTE",keep="first")

# Reemplazar valores nulos con "No especificado" en las columnas seleccionadas
dim_estudiantes_magister = dim_estudiantes_magister.with_columns([
    pl.col(col_name).fill_null("No especificado") for col_name in [
        "TIPO_ESTUDIANTE_STDN", "DESCRIPCION_TIPO_ESTUDIANTE_STDN", "TIPO_ADMISION", "DESCRIPCION_TIPO_ADMISION", "ESTADO_ACADEMICO_INICIO_SEMESTRE", "CIUDAD_NACIMIENTO", "NACIONALIDAD", "DESCRIPCION_NACIONALIDAD", "EDAD", "SEXO", "DESCRIPCION_SEXO", "ESTADO_CIVIL", "DESCRIPCION_ESTADO_CIVIL", "COHORTE", "COHORTE_DESCRIPCION", "DIRECCION", "CIUDAD", "CODIGO_CIUDAD", "CODIGO_DEPARTAMENTO", "CODIGO_PAIS", "INDICATIVO", "TELEFONO", "EXTENSION", "CELULAR", "CORREO_PERSONAL", "CORREO_UNIANDES", "ALERTAS", "ESTADO_ENTREGA_DOCUMENTOS_PG", "OBSERVACIONES_DOCUMENTOS_ENTREGADOS_PG", "ESTRATO", "CODIGO_COLEGIO", "CODIGO_DANE", "NOMBRE_COLEGIO", "CIUDAD_COLEGIO", "DEPARTAMENTO_COLEGIO", "TIPO_COLEGIO", "PEN_CURSO", "PEN_POSC", "PEN_TAM", "PEN_PGA", "UL_CURSO", "UL_POSC", "UL_TAM", "UL_PGA", "SE_HA_PRESENTADO", "SE_HA_MATRICULADO", "SOLICITA_APOYO_FINANCIERO", "TIPO_EXAMEN", "SNP", "PUNTAJE", "PLC", "PMA", "PSC", "PCN", "PIN", "EXAMEN_INTERNO", "EXAMEN_INTERNO_NOTA", "EXAMEN_HOMOLOGABLE", "PREGRADO_QS", "PREGRADO_QS_UNI", "PREGRADO_QS_VERFICADO", "APELLIDOS_PADRE", "NOMBRES_PADRE", "DIRECCION_PADRE", "CIUDAD_PADRE", "INDICATIVO_PADRE", "TELEFONO_PADRE", "EXTENSION_TELEFONO_PADRE", "CORREO_PADRE", "APELLIDOS_MADRE", "NOMBRES_MADRE", "DIRECCION_MADRE", "CIUDAD_MADRE", "INDICATIVO_MADRE", "TELEFONO_MADRE", "EXTENSION_TELEFONO_MADRE", "CORREO_MADRE", "APELLIDOS_ACUDIENTE", "NOMBRES_ACUDIENTE", "DIRECCION_ACUDIENTE", "CIUDAD_ACUDIENTE", "INDICATIVO_ACUDIENTE", "TELEFONO_ACUDIENTE", "EXTENSION_TELEFONO_ACUDIENTE", "CORREO_ACUDIENTE", "NUMERO_HERMANOS", "POSICION_ENTRE_HERMANOS", "ESTADO_CIVIL_PADRES", "NIVEL_EDUCATIVO_MADRE", "OCUPACION_MADRE", "NIVEL_EDUCATIVO_PADRE", "OCUPACION_PADRE", "TIPO_VIVIENDA", "CURSO_BACH_UN_COLEGIO", "NUMERO_COLEGIOS_BACH", "PERTENECE_MINORIA_ETNICA", "SISBEN", "PUN_SISBEN", "CUMPLE_I1", "CUMPLE_EI", "CUMPLE_RANK", "CUMPLE_PGA", "CUMPLE_UNAL", "CUMPLE_QS", "FORMAS_ADMISION", "INDICADOR_GRADUADO", "PROMEDIO_ACUMULADO", "CONOCIMIENTOS_DEL_PROGRAMA", "FORMA_DE_FINANCIACION", "REQUIERE_APOYO_FINANCIERO", "LENGUA_MATERNA", "NIVEL_INGLES_LECTURA", "NIVEL_INGLES_ESCRITO", "NIVEL_INGLES_ORAL", "PORCENTAJE_TIEMPO_PARA_ESTUDIO", "MATRICULADO_BANNER", "MATRICULADO_FINANCIERA", "TIPO_FINANCIERA", "TOTAL_PAGO_FINANCIERA", "MANEJO_DATOS_SENSIBLES", "EDAD_ADM", "FECHA_GRADO_PS", "ESTADO_ACADEMICO_COMBINADO", "INSCRITO"
    ]
])

# Resultado final con valores nulos reemplazados

In [0]:
# dim_periodo

# Seleccionar y eliminar duplicados en la columna 'PERIODO'
dim_periodo = Info_Notas_HA_Actual.select("PERIODO").unique()

# Crear columnas duplicadas para los primeros y últimos caracteres
dim_periodo = (
    dim_periodo.with_columns([
        pl.col("PERIODO").alias("ANIO_PERIODO"),
        pl.col("PERIODO").str.slice(-2).alias("PERIODO"),
        pl.col("PERIODO").str.slice(0, 4).alias("ANIO")
    ])
)

# Agregar columnas "FECHA_INICIO" y "FECHA_FIN" basadas en el valor de "PERIODO"
dim_periodo = dim_periodo.with_columns([
    pl.when(pl.col("PERIODO") == "10").then(pl.date(pl.col("ANIO").cast(pl.Int32), 1, 1))
      .when(pl.col("PERIODO") == "20").then(pl.date(pl.col("ANIO").cast(pl.Int32), 8, 1))
      .when(pl.col("PERIODO") == "19").then(pl.date(pl.col("ANIO").cast(pl.Int32), 6, 1))
      .otherwise(None).alias("FECHA_INICIO"),
    pl.when(pl.col("PERIODO") == "10").then(pl.date(pl.col("ANIO").cast(pl.Int32), 5, 31))
      .when(pl.col("PERIODO") == "20").then(pl.date(pl.col("ANIO").cast(pl.Int32), 12, 1))
      .when(pl.col("PERIODO") == "19").then(pl.date(pl.col("ANIO").cast(pl.Int32), 7, 31))
      .otherwise(None).alias("FECHA_FIN")
])

In [0]:
#dim_profes

# Seleccionar columnas específicas
dim_profes = docentes_cursos.select([
    "CODIGO_DOCENTE", "INDICADOR_DOCENTE", "INDICADOR_ASESOR", "IDENTIFICACION", "FACULTADES", "CODIGO_FACULTAD"
])

# Eliminar duplicados basado en la columna CODIGO_DOCENTE
dim_profes = dim_profes.unique(subset=["CODIGO_DOCENTE"])

# Cargar otra tabla para hacer la combinación (ejemplo de carga)
# info_personas = pl.read_parquet("ruta_a_info_personas.parquet")

# Hacer combinación izquierda con info_personas en base a CODIGO_DOCENTE y CODIGO_ESTUDIANTE
dim_profes = dim_profes.join(
    info_persona, left_on="CODIGO_DOCENTE", right_on="CODIGO_ESTUDIANTE", how="left"
)

# Expandir columnas anidadas de la combinación
expand_columns = [
    "PRIMER_NOMBRE", "SEGUNDO_NOMBRE", "NOMBRES", "PRIMER_APELLIDO", "SEGUNDO_APELLIDO",
    "APELLIDOS", "PRIMER_NOMBRE_IDENTITARIO", "SEGUNDO_NOMBRE_IDENTITARIO",
    "PRIMER_APELLIDO_IDENTITARIO", "SEGUNDO_APELLIDO_IDENTITARIO", "CODIGO_TIPO_IDENTIFICACION",
    "TIPO_IDENTIFICACION", "IDENTIFICACION", "TIPO_GENERO", "GENERO", "CODIGO_TIPO_NACIONALIDAD",
    "NACIONALIDAD", "FECHA_DE_NACIMIENTO", "EDAD", "CODIGO_TIPO_ESTADO_CIVIL", "TIPO_ESTADO_CIVIL",
    "FALLECIDO", "FECHA_DE_FALLECIMIENTO", "LOGIN", "CORREO_INSTITUCIONAL", "CORREO_PERSONAL",
    "TELEFONO_RESIDENCIA", "TELEFONO_CELULAR", "TELEFONO_TRABAJO", "DIRECCION_RESIDENCIA",
    "CODIGO_CIUDAD_RESIDENCIA", "CIUDAD_RESIDENCIA", "CODIGO_DEPARTAMENTO_RESIDENCIA",
    "DEPARTAMENTO_RESIDENCIA", "CODIGO_PAIS_RESIDENCIA", "PAIS_RESIDENCIA", "DIRECCION_TRABAJO",
    "CODIGO_CIUDAD_TRABAJO", "CIUDAD_TRABAJO", "CODIGO_DEPARTAMENTO_TRABAJO", "DEPARTAMENTO_TRABAJO",
    "CODIGO_PAIS_TRABAJO", "PAIS_TRABAJO", "NOMBRE_PADRE", "TELEFONO_PADRE", "NOMBRE_MADRE",
    "TELEFONO_MADRE", "REPRESENTANTE_FINANCIERO", "DIRECCION_REPRESENTANTE_FINANCIERO",
    "TELEFONO_REPRESENTANTE_FINANCIERO", "CORREO_REPRESENTANTE_FINANCIERO", "INDICADOR_CONFIDENCIALIDAD",
    "CIUDAD_RESIDENCIA_UG", "DEPARTAMENTO_RESIDENCIA_UG", "PAIS_RESIDENCIA_UG", "CIUDAD_TRABAJO_UG",
    "DEPARTAMENTO_TRABAJO_UG", "PAIS_TRABAJO_UG", "CIUDAD_NACIMIENTO", "ESTRATO", "FACULTADES"
]
# dim_profes = dim_profes.select(expand_columns)

dim_profes = dim_profes.with_columns([
    pl.col(col_name).alias("info_personas."+col_name) for col_name in expand_columns])

# Reemplazar valores nulos con "No especificado" en varias columnas

replace_cols = [
    "PRIMER_NOMBRE", "SEGUNDO_NOMBRE", "NOMBRES", "PRIMER_APELLIDO", "SEGUNDO_APELLIDO", "APELLIDOS", "PRIMER_NOMBRE_IDENTITARIO", "SEGUNDO_NOMBRE_IDENTITARIO", "PRIMER_APELLIDO_IDENTITARIO", "SEGUNDO_APELLIDO_IDENTITARIO", "CODIGO_TIPO_IDENTIFICACION", "TIPO_IDENTIFICACION", "IDENTIFICACION", "TIPO_GENERO", "GENERO", "CODIGO_TIPO_NACIONALIDAD", "NACIONALIDAD", "FECHA_DE_NACIMIENTO", "EDAD", "CODIGO_TIPO_ESTADO_CIVIL", "TIPO_ESTADO_CIVIL", "FALLECIDO", "FECHA_DE_FALLECIMIENTO", "LOGIN", "CORREO_INSTITUCIONAL", "CORREO_PERSONAL", "TELEFONO_RESIDENCIA", "TELEFONO_CELULAR", "TELEFONO_TRABAJO", "DIRECCION_RESIDENCIA", "CODIGO_CIUDAD_RESIDENCIA", "CIUDAD_RESIDENCIA", "CODIGO_DEPARTAMENTO_RESIDENCIA", "DEPARTAMENTO_RESIDENCIA", "CODIGO_PAIS_RESIDENCIA", "PAIS_RESIDENCIA", "DIRECCION_TRABAJO", "CODIGO_CIUDAD_TRABAJO", "CIUDAD_TRABAJO", "CODIGO_DEPARTAMENTO_TRABAJO", "DEPARTAMENTO_TRABAJO", "CODIGO_PAIS_TRABAJO", "PAIS_TRABAJO", "NOMBRE_PADRE", "TELEFONO_PADRE", "NOMBRE_MADRE", "TELEFONO_MADRE", "REPRESENTANTE_FINANCIERO", "DIRECCION_REPRESENTANTE_FINANCIERO", "TELEFONO_REPRESENTANTE_FINANCIERO", "CORREO_REPRESENTANTE_FINANCIERO", "INDICADOR_CONFIDENCIALIDAD", "CIUDAD_RESIDENCIA_UG", "DEPARTAMENTO_RESIDENCIA_UG", "PAIS_RESIDENCIA_UG", "CIUDAD_TRABAJO_UG", "DEPARTAMENTO_TRABAJO_UG", "PAIS_TRABAJO_UG", "CIUDAD_NACIMIENTO", "ESTRATO", "FACULTADES","info_personas.PRIMER_NOMBRE", "info_personas.SEGUNDO_NOMBRE", "info_personas.NOMBRES", "info_personas.PRIMER_APELLIDO", "info_personas.SEGUNDO_APELLIDO", "info_personas.APELLIDOS", "info_personas.PRIMER_NOMBRE_IDENTITARIO", "info_personas.SEGUNDO_NOMBRE_IDENTITARIO", "info_personas.PRIMER_APELLIDO_IDENTITARIO", "info_personas.SEGUNDO_APELLIDO_IDENTITARIO", "info_personas.CODIGO_TIPO_IDENTIFICACION", "info_personas.TIPO_IDENTIFICACION", "info_personas.IDENTIFICACION", "info_personas.TIPO_GENERO", "info_personas.GENERO", "info_personas.CODIGO_TIPO_NACIONALIDAD", "info_personas.NACIONALIDAD", "info_personas.FECHA_DE_NACIMIENTO", "info_personas.EDAD", "info_personas.CODIGO_TIPO_ESTADO_CIVIL", "info_personas.TIPO_ESTADO_CIVIL", "info_personas.FALLECIDO", "info_personas.FECHA_DE_FALLECIMIENTO", "info_personas.LOGIN", "info_personas.CORREO_INSTITUCIONAL", "info_personas.CORREO_PERSONAL", "info_personas.TELEFONO_RESIDENCIA", "info_personas.TELEFONO_CELULAR", "info_personas.TELEFONO_TRABAJO", "info_personas.DIRECCION_RESIDENCIA", "info_personas.CODIGO_CIUDAD_RESIDENCIA", "info_personas.CIUDAD_RESIDENCIA", "info_personas.CODIGO_DEPARTAMENTO_RESIDENCIA", "info_personas.DEPARTAMENTO_RESIDENCIA", "info_personas.CODIGO_PAIS_RESIDENCIA", "info_personas.PAIS_RESIDENCIA", "info_personas.DIRECCION_TRABAJO", "info_personas.CODIGO_CIUDAD_TRABAJO", "info_personas.CIUDAD_TRABAJO", "info_personas.CODIGO_DEPARTAMENTO_TRABAJO", "info_personas.DEPARTAMENTO_TRABAJO", "info_personas.CODIGO_PAIS_TRABAJO", "info_personas.PAIS_TRABAJO", "info_personas.NOMBRE_PADRE", "info_personas.TELEFONO_PADRE", "info_personas.NOMBRE_MADRE", "info_personas.TELEFONO_MADRE", "info_personas.REPRESENTANTE_FINANCIERO", "info_personas.DIRECCION_REPRESENTANTE_FINANCIERO", "info_personas.TELEFONO_REPRESENTANTE_FINANCIERO", "info_personas.CORREO_REPRESENTANTE_FINANCIERO", "info_personas.INDICADOR_CONFIDENCIALIDAD", "info_personas.CIUDAD_RESIDENCIA_UG", "info_personas.DEPARTAMENTO_RESIDENCIA_UG", "info_personas.PAIS_RESIDENCIA_UG", "info_personas.CIUDAD_TRABAJO_UG", "info_personas.DEPARTAMENTO_TRABAJO_UG", "info_personas.PAIS_TRABAJO_UG", "info_personas.CIUDAD_NACIMIENTO", "info_personas.ESTRATO", "info_personas.FACULTADES","INDICADOR_DOCENTE","INDICADOR_ASESOR","CODIGO_FACULTAD",
    "IDENTIFICACION_right","FACULTADES_right","CODIGO_FACULTAD_right"]
    
dim_profes = dim_profes.with_columns([pl.col(col_name).fill_null("No especificado") for col_name in replace_cols])

In [0]:
gc.collect()  # Llamar explícitamente al garbage collector

13812

In [0]:
# # dim_seccion

# dim_seccion = Info_Notas_HA_Actual.select(["SECCION","CRN"])
# dim_seccion = dim_seccion.unique(subset=["SECCION","CRN"])


In [0]:
# hechos_estudiante_curso

# Reemplazar valores en columna "MATERIA"
hechos_estudiante_curso = Info_Notas_HA_Actual.with_columns(pl.col("MATERIA").str.replace("-Homologada", ""))

# Seleccionar columnas específicas
columns_to_select = [
    "PERIODO", "CODIGO_ESTUDIANTE", "SECCION", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL", 
    "ESTATUS_CURSO", "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTADO_MATERIA", 
    "DESCRIPCION_ESTADO_MATERIA", "MATERIA","CRN", "CODIGO_DOCENTE", "CODIGO_NIVEL_PROGRAMA_1", 
    "CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_2", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_3", 
    "CODIGO_PROGRAMA_3"
]
hechos_estudiante_curso = hechos_estudiante_curso.select(columns_to_select)

# Eliminar duplicados en las columnas seleccionadas
hechos_estudiante_curso = hechos_estudiante_curso.unique(subset=["PERIODO", "CODIGO_ESTUDIANTE", "MATERIA"])

hechos_estudiante_curso = hechos_estudiante_curso.join(dim_union_grupo_horario.select(["PERIODO","CRN","ID_GRUPO_HORARIO"]), on=["PERIODO","CRN"], how="left")

hechos_estudiante_curso = hechos_estudiante_curso.unique(subset=["CODIGO_ESTUDIANTE","MATERIA","PERIODO","CRN"])



# hechos_estudiante_curso= hechos_estudiante_curso.join(dim_horario, on=["FECHA_INICIO_FRANJA", "FECHA_FIN_FRANJA", "HORA_INICIO_FRANJA", "HORA_FIN_FRANJA", "LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO", "DOMINGO"],how="left")

# Realizar joins con otras tablas (simulado aquí, necesitas ajustar según tus tablas)
hechos_estudiante_curso = hechos_estudiante_curso.join(dim_curso.select(["ID_MATERIA", "HORAS_CREDITO","MATERIA"]), on="MATERIA", how="left")
hechos_estudiante_curso = hechos_estudiante_curso.with_columns([pl.col("ID_MATERIA").alias("dim_curso.ID_MATERIA"),
                                                                pl.col("HORAS_CREDITO").alias("dim_curso.HORAS_CREDITO"),
                                                                pl.col("MATERIA").alias("dim_curso.MATERIA")])

# Seleccionar columnas específicas
columns_to_select = [
    "PERIODO", "CODIGO_ESTUDIANTE", "SECCION", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL", "ESTATUS_CURSO", "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTADO_MATERIA", "DESCRIPCION_ESTADO_MATERIA", "CODIGO_DOCENTE", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_2", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_3", "CODIGO_PROGRAMA_3", "dim_curso.ID_MATERIA", "dim_curso.HORAS_CREDITO","CRN","dim_curso.MATERIA","ID_GRUPO_HORARIO"
]
hechos_estudiante_curso = hechos_estudiante_curso.select(columns_to_select)

columns_to_fill = ["CODIGO_PROGRAMA_1","CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2","CODIGO_NIVEL_PROGRAMA_2", "CODIGO_PROGRAMA_3","CODIGO_NIVEL_PROGRAMA_3"]

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([pl.col(col).str.strip_chars() for col in columns_to_fill])

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([pl.col(col).fill_null("No especificado") for col in columns_to_fill])

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([
    pl.when(pl.col(col) == "").then(pl.lit("No especificado")).otherwise(pl.col(col)).alias(col) 
    for col in columns_to_fill
])

hechos_estudiante_curso = hechos_estudiante_curso.join(dim_programa.select(["ID_PROGRAMA","CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2", "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"]), on=["CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2", "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"], how="left")

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([pl.col("ID_PROGRAMA").alias("dim_programa.ID_PROGRAMA")])

# Reemplazar valores nulos y vacíos
replace_columns = [
    "CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"
]

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([pl.col(col).str.strip_chars() for col in replace_columns])

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([pl.col(col).fill_null("No especificado") for col in replace_columns])

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([
    pl.when(pl.col(col) == "").then(pl.lit("No especificado")).otherwise(pl.col(col)).alias(col) 
    for col in replace_columns
])

#selccionar columnas

columnas_filtro=['PERIODO',
 'CODIGO_ESTUDIANTE',
 'SECCION',
 'CALIFICACION_PARCIAL',
 'CALIFICACION_FINAL',
 'ESTATUS_CURSO',
 'CALIFICACION_PARCIAL_CONTEO',
 'CALIFICACION_FINAL_CONTEO',
 'ESTADO_MATERIA',
 'DESCRIPCION_ESTADO_MATERIA',
 'CODIGO_DOCENTE',
 'CODIGO_NIVEL_PROGRAMA_1',
 'CODIGO_PROGRAMA_1',
 'CODIGO_NIVEL_PROGRAMA_2',
 'CODIGO_PROGRAMA_2',
 'CODIGO_NIVEL_PROGRAMA_3',
 'CODIGO_PROGRAMA_3',
 'dim_curso.ID_MATERIA',
 'dim_curso.HORAS_CREDITO',
 'dim_programa.ID_PROGRAMA',
 "CRN",
 "dim_curso.MATERIA",
 "ID_GRUPO_HORARIO"]

# Concatenar por filas
hechos_estudiante_curso = pl.concat([hechos_estudiante_curso.select(columnas_filtro), info_retiros_201920_hacia_atras.select(columnas_filtro)], how="vertical")

replace_columns = [
    "PERIODO", "CODIGO_ESTUDIANTE", "SECCION", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL", "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTADO_MATERIA", "DESCRIPCION_ESTADO_MATERIA", "CODIGO_DOCENTE", "dim_curso.ID_MATERIA", "dim_curso.HORAS_CREDITO", "dim_programa.ID_PROGRAMA","ID_GRUPO_HORARIO","CRN"
]

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([pl.col(col).fill_null("No especificado") for col in replace_columns])

# Filtrar por el valor de "PERIODO" distinto de "999999"
hechos_estudiante_curso = hechos_estudiante_curso.filter(pl.col("PERIODO") != "999999")

hechos_estudiante_curso = hechos_estudiante_curso.with_columns([
    pl.when(pl.col("DESCRIPCION_ESTADO_MATERIA").str.contains("NO PAGO"))
    .then(pl.lit("RETIRADO NO PAGO"))
    .when(pl.col("DESCRIPCION_ESTADO_MATERIA").str.contains("RETIR"))
    .then(pl.lit("RETIRADO"))
    .otherwise(pl.lit("NO RETIRADO"))
    .alias("RETIRO")
])

valores_excluir = ["R", "A", "TR", "I", "RETIRADO", "P", "PE", "PD", "NT", "No especificado"]

# Crear columna "NOTA_FINAL_MOD" basada en "CALIFICACION_FINAL"
hechos_estudiante_curso = hechos_estudiante_curso.with_columns(
    pl.when(pl.col("CALIFICACION_FINAL") == "R").then(1)
    .when(pl.col("CALIFICACION_FINAL").is_in(["A", "TR"])).then(4)
    .when(pl.col("CALIFICACION_FINAL") == "I").then(-1)
    .when(pl.col("RETIRO") == "RETIRADO").then(-2)
    .when(pl.col("CALIFICACION_FINAL").is_in(["P", "PE", "PD"])).then(-3)
    .when(pl.col("CALIFICACION_FINAL") == "NT").then(-4)
    .when(pl.col("CALIFICACION_FINAL") == "No especificado").then(-5)
    .when((~pl.col("CALIFICACION_FINAL").cast(pl.Utf8).is_in(valores_excluir)) & (~pl.col("CALIFICACION_FINAL").cast(pl.Utf8).str.contains(r"\."))).then(-6)
    .otherwise(pl.col("CALIFICACION_FINAL"))
    .cast(pl.Float64)
    .alias("NOTA_FINAL_MOD")
)

# Crear columna "ESTADO_MATERIA_FINAL" basada en "NOTA_FINAL_MOD"
hechos_estudiante_curso = hechos_estudiante_curso.with_columns([
    pl.when(pl.col("NOTA_FINAL_MOD") >= 3).then(pl.lit("APROBADO"))
    .when(pl.col("NOTA_FINAL_MOD") == -1).then(pl.lit("INCOM"))
    .when(pl.col("NOTA_FINAL_MOD") == -2).then(pl.lit("RETIRADO"))
    .when(pl.col("NOTA_FINAL_MOD") == -3).then(pl.lit("PENDIENTE"))
    .when(pl.col("NOTA_FINAL_MOD") == -4).then(pl.lit("NO TERMINADO"))
    .when(pl.col("NOTA_FINAL_MOD") == -5).then(pl.lit("NO ESPECIFICADO"))
    .when(pl.col("NOTA_FINAL_MOD") == -6).then(pl.lit("EXAMEN"))
    .otherwise(pl.lit("REPROBADO"))  # Aquí aseguramos que sea un literal de texto
    .alias("ESTADO_MATERIA_FINAL")
])

# Cambiar el tipo de dato de "PERIODO" a int
hechos_estudiante_curso = hechos_estudiante_curso.with_columns(pl.col("PERIODO").cast(pl.Int64))


# Seleccionar las columnas finales
hechos_estudiante_curso= hechos_estudiante_curso.select(["PERIODO", "CODIGO_ESTUDIANTE", "SECCION", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL", "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTATUS_CURSO", "RETIRO", "NOTA_FINAL_MOD", "ESTADO_MATERIA_FINAL", "dim_curso.ID_MATERIA","CRN", "dim_curso.HORAS_CREDITO", "dim_programa.ID_PROGRAMA",'CODIGO_NIVEL_PROGRAMA_1','CODIGO_NIVEL_PROGRAMA_2','CODIGO_NIVEL_PROGRAMA_3','CODIGO_DOCENTE',"dim_curso.MATERIA","ID_GRUPO_HORARIO"])


hechos_estudiante_curso = hechos_estudiante_curso.with_columns(
    pl.when(pl.col("dim_curso.MATERIA").str.contains("LENG-"))
      .then(2)  # Si contiene "LENG-", asignar 2
      .otherwise(pl.col("dim_curso.HORAS_CREDITO"))  # Si no, mantener el valor original
      .alias("dim_curso.HORAS_CREDITO")  # Renombrar a la misma columna
)

agrupacion_creditos = hechos_estudiante_curso

hechos_estudiante_curso = hechos_estudiante_curso.select(["PERIODO", "CODIGO_ESTUDIANTE", "SECCION", "CALIFICACION_PARCIAL", "CALIFICACION_FINAL", "CALIFICACION_PARCIAL_CONTEO", "CALIFICACION_FINAL_CONTEO", "ESTATUS_CURSO", "RETIRO", "NOTA_FINAL_MOD", "ESTADO_MATERIA_FINAL", "dim_curso.ID_MATERIA","CRN", "dim_curso.HORAS_CREDITO", "dim_programa.ID_PROGRAMA",'CODIGO_NIVEL_PROGRAMA_1','CODIGO_NIVEL_PROGRAMA_2','CODIGO_NIVEL_PROGRAMA_3','CODIGO_DOCENTE',"ID_GRUPO_HORARIO"])

In [0]:
# dim_tipo_financiamiento

# Paso 2: Seleccionar la columna "SECCION_MOD" y eliminar duplicados.
dim_tipo_financiamiento = matriculas_pago_base_becas.select("SECCION_MOD").unique()

# Paso 3: Agregar una columna de índice.
dim_tipo_financiamiento = dim_tipo_financiamiento.with_row_index(name="Índice")

# Paso 4: Reordenar las columnas.
dim_tipo_financiamiento = dim_tipo_financiamiento.select(["Índice", "SECCION_MOD"])


In [0]:
#ID_SECCION_PUNTAJE

# Paso 2: Seleccionar las columnas necesarias
id_seccion_puntaje = matriculas_pago_base_becas.select(["PERIODO", "CODIGO_ESTUDIANTE", "SECCION_MOD", "PORCENTAJE_APORTE_ITEM", "TOTAL_PAGADO", "ranking"])

# Paso 3: Crear columna combinada "PERIODO_CODIGO"
id_seccion_puntaje = id_seccion_puntaje.with_columns(
    (pl.col("PERIODO").cast(str) + pl.col("CODIGO_ESTUDIANTE").cast(str)).alias("PERIODO_CODIGO")
)

# Paso 4: Crear la columna "SECCION_PORCENTAJE" combinando "SECCION_MOD" y "PORCENTAJE_APORTE_ITEM"
id_seccion_puntaje = id_seccion_puntaje.with_columns(
    (pl.col("SECCION_MOD") + "-" + pl.col("PORCENTAJE_APORTE_ITEM").cast(str)).alias("SECCION_PORCENTAJE")
)

# Paso 5: Seleccionar la columna "SECCION_PORCENTAJE"
id_seccion_puntaje = id_seccion_puntaje.select("SECCION_PORCENTAJE")

# Paso 6: Eliminar duplicados en "SECCION_PORCENTAJE"
id_seccion_puntaje = id_seccion_puntaje.unique()

# Paso 7: Agregar un índice como "ID_SECCION_PUNTAJE"
id_seccion_puntaje = id_seccion_puntaje.with_row_index(name="ID_SECCION_PUNTAJE", offset=1)

# Paso 8: Reordenar columnas si es necesario
id_seccion_puntaje = id_seccion_puntaje.select(["ID_SECCION_PUNTAJE", "SECCION_PORCENTAJE"])


In [0]:
#generacion_ids_grupo

# Paso 2: Seleccionar columnas específicas y eliminar la columna no deseada
generacion_ids_grupo = matriculas_pago_base_becas.select(["PERIODO", "CODIGO_ESTUDIANTE", "SECCION_MOD", "PORCENTAJE_APORTE_ITEM", "TOTAL_PAGADO", "ranking"])

# Paso 3: Crear la columna combinada "PERIODO_CODIGO"
generacion_ids_grupo = generacion_ids_grupo.with_columns(
    (pl.col("PERIODO").cast(str) + pl.col("CODIGO_ESTUDIANTE").cast(str)).alias("PERIODO_CODIGO")
)

# Paso 4: Crear la columna "Combinacion"
generacion_ids_grupo = generacion_ids_grupo.with_columns(
    (pl.col("SECCION_MOD") + "-" + pl.col("PORCENTAJE_APORTE_ITEM").cast(str)).alias("SECCION_PORCENTAJE")
)

# Paso 5: Cargar y unir la tabla auxiliar ID_SECCION_PUNTAJE
# (Aquí se asume que ID_SECCION_PUNTAJE está disponible en el mismo formato para unir)
# id_seccion_puntaje = id_seccion_puntaje.rename({"SECCION_PORCENTAJE": "Combinacion"})

generacion_ids_grupo = generacion_ids_grupo.join(id_seccion_puntaje.select(["ID_SECCION_PUNTAJE", "SECCION_PORCENTAJE"]), on="SECCION_PORCENTAJE", how="left")

# Paso 6: Agrupar por "PERIODO_CODIGO" y agregar como lista los valores de ID_SECCION_PUNTAJE
generacion_ids_grupo = generacion_ids_grupo.group_by("PERIODO_CODIGO").agg(
    pl.col("ID_SECCION_PUNTAJE").str.concat("").alias("ID_GRUPO")
)
# print(generacion_ids_grupo)


In [0]:
# dim_grupo

# Paso 2: Seleccionar las columnas necesarias
dim_grupo = matriculas_pago_base_becas.select(["PERIODO", "CODIGO_ESTUDIANTE", "SECCION_MOD", "PORCENTAJE_APORTE_ITEM", "TOTAL_PAGADO", "ranking"])

# Paso 3: Crear columna combinada "PERIODO_CODIGO"
dim_grupo = dim_grupo.with_columns(
    (pl.col("PERIODO").cast(str) + pl.col("CODIGO_ESTUDIANTE").cast(str)).alias("PERIODO_CODIGO")
)

# Paso 4: Crear columna "Combinacion" concatenando "SECCION_MOD" y "PORCENTAJE_APORTE_ITEM"
dim_grupo = dim_grupo.with_columns(
    (pl.col("SECCION_MOD") + "-" + pl.col("PORCENTAJE_APORTE_ITEM").cast(str)).alias("SECCION_PORCENTAJE")
)

# Paso 5: Realizar el join con la tabla 'generacion_ids_grupo' en la columna "PERIODO_CODIGO"
# Nota: Esta línea requiere que tengas la tabla `generacion_ids_grupo` cargada como un DataFrame en `df_generacion_ids`
dim_grupo = dim_grupo.join(generacion_ids_grupo, on="PERIODO_CODIGO", how="left")

# Paso 6: Seleccionar columnas específicas después del join
dim_grupo = dim_grupo.select(["SECCION_MOD", "PORCENTAJE_APORTE_ITEM", "SECCION_PORCENTAJE", "ID_GRUPO"])

# Paso 7: Reordenar las columnas
dim_grupo = dim_grupo.select(["ID_GRUPO", "SECCION_PORCENTAJE", "SECCION_MOD", "PORCENTAJE_APORTE_ITEM"])

# Paso 8: Eliminar duplicados en las columnas "ID_GRUPO" y "SECCION_PORCENTAJE"
dim_grupo = dim_grupo.unique(subset=["ID_GRUPO"])

# Paso 9: 
dim_grupo = dim_grupo.select(["ID_GRUPO"])

In [0]:
# dim_union_grupo_tipo

# Seleccionar columnas específicas
dim_union_grupo_tipo = matriculas_pago_base_becas.select(["PERIODO", "CODIGO_ESTUDIANTE", "SECCION_MOD", "PORCENTAJE_APORTE_ITEM", "TOTAL_PAGADO", "ranking"])

# Crear la columna 'PERIODO_CODIGO' combinando 'PERIODO' y 'CODIGO_ESTUDIANTE' sin delimitador
dim_union_grupo_tipo = dim_union_grupo_tipo.with_columns(
    (pl.col("PERIODO").cast(str) + pl.col("CODIGO_ESTUDIANTE").cast(str)).alias("PERIODO_CODIGO")
)

# Crear la columna 'SECCION_PORCENTAJE' combinando 'SECCION_MOD' y 'PORCENTAJE_APORTE_ITEM' con un delimitador "-"
dim_union_grupo_tipo = dim_union_grupo_tipo.with_columns(
    (pl.col("SECCION_MOD") + "-" + pl.col("PORCENTAJE_APORTE_ITEM").cast(str)).alias("SECCION_PORCENTAJE")
)

# Realizar el join con la tabla 'generacion_ids_grupo' en la columna 'PERIODO_CODIGO'
# Asegúrate de cargar 'generacion_ids_grupo' como un DataFrame de Polars también

dim_union_grupo_tipo = dim_union_grupo_tipo.join(generacion_ids_grupo, on="PERIODO_CODIGO", how="left").select([
    "SECCION_MOD", "PORCENTAJE_APORTE_ITEM", "SECCION_PORCENTAJE", "ID_GRUPO"
])

# Reordenar columnas
dim_union_grupo_tipo = dim_union_grupo_tipo.select(["ID_GRUPO", "SECCION_PORCENTAJE", "SECCION_MOD", "PORCENTAJE_APORTE_ITEM"])

# Eliminar duplicados en 'ID_GRUPO' y 'SECCION_PORCENTAJE'
dim_union_grupo_tipo = dim_union_grupo_tipo.unique(subset=["ID_GRUPO", "SECCION_PORCENTAJE"])



In [0]:
#AGRUPACION CREDITOS

# Reemplazar valores vacíos y nulos en la columna 'dim_curso.HORAS_CREDITO' por '0'

agrupacion_creditos= agrupacion_creditos.filter(~pl.col("dim_curso.MATERIA").str.contains("DEPO"))

# Crear columnas para CREDITOS_APROBADOS, CREDITOS_REPROBADOS, CREDITOS_RETIRADOS, y CREDITOS_INCOMPLETOS
agrupacion_creditos = agrupacion_creditos.with_columns([
    pl.when(pl.col("ESTADO_MATERIA_FINAL") == "APROBADO").then(pl.col("dim_curso.HORAS_CREDITO")).otherwise(pl.when(pl.col("dim_curso.HORAS_CREDITO") == "No especificado").then(0).otherwise(0)).alias("CREDITOS_APROBADOS"),
    pl.when(pl.col("ESTADO_MATERIA_FINAL") == "REPROBADO").then(pl.col("dim_curso.HORAS_CREDITO")).otherwise(pl.when(pl.col("dim_curso.HORAS_CREDITO") == "No especificado").then(0).otherwise(0)).alias("CREDITOS_REPROBADOS"),
    pl.when(pl.col("ESTADO_MATERIA_FINAL") == "RETIRADO").then(pl.col("dim_curso.HORAS_CREDITO")).otherwise(pl.when(pl.col("dim_curso.HORAS_CREDITO") == "No especificado").then(0).otherwise(0)).alias("CREDITOS_RETIRADOS"),
    pl.when(pl.col("ESTADO_MATERIA_FINAL") == "INCOM").then(pl.col("dim_curso.HORAS_CREDITO")).otherwise(pl.when(pl.col("dim_curso.HORAS_CREDITO") == "No especificado").then(0).otherwise(0)).alias("CREDITOS_INCOMPLETOS")
])

# Reemplazar 'No especificado' por 0 en las nuevas columnas
agrupacion_creditos = agrupacion_creditos.with_columns([
    pl.when(pl.col(col) == "No especificado").then(0).otherwise(pl.col(col)).alias(col)
    for col in ["CREDITOS_APROBADOS", "CREDITOS_REPROBADOS", "CREDITOS_RETIRADOS", "CREDITOS_INCOMPLETOS","dim_curso.HORAS_CREDITO"]
])

# Cambiar tipos de las columnas a número (primero a float y luego a int)
agrupacion_creditos = agrupacion_creditos.with_columns([
    pl.col("CREDITOS_APROBADOS").cast(pl.Float64).cast(pl.Int32),
    pl.col("CREDITOS_REPROBADOS").cast(pl.Float64).cast(pl.Int32),
    pl.col("CREDITOS_RETIRADOS").cast(pl.Float64).cast(pl.Int32),
    pl.col("CREDITOS_INCOMPLETOS").cast(pl.Float64).cast(pl.Int32),
    pl.col("dim_curso.HORAS_CREDITO").cast(pl.Float64).cast(pl.Int32)
])

# Agrupar filas por 'PERIODO' y 'CODIGO_ESTUDIANTE', sumando las columnas correspondientes
agrupacion_creditos = agrupacion_creditos.group_by(["PERIODO", "CODIGO_ESTUDIANTE"]).agg([
    pl.col("CREDITOS_APROBADOS").sum().alias("CREDITOS_APROBADOS"),
    pl.col("CREDITOS_REPROBADOS").sum().alias("CREDITOS_REPROBADOS"),
    pl.col("CREDITOS_RETIRADOS").sum().alias("CREDITOS_RETIRADOS"),
    pl.col("CREDITOS_INCOMPLETOS").sum().alias("CREDITOS_INCOMPLETOS"),
    pl.col("dim_curso.HORAS_CREDITO").sum().alias("TOTAL_CREDITOS")
])

agrupacion_creditos = agrupacion_creditos.with_columns(pl.col("PERIODO").cast(str))

In [0]:
#ESTUDIANTE GRADUADO PREGRADO, MAESTRIA, DOCTORADO?
graduados_pregrado = graduados.filter(pl.col("NIVEL_CODE") == "PR") 
graduados_maestria = graduados.filter((pl.col("NIVEL_CODE") == "MA") | (pl.col("NIVEL_CODE") == "ES"))
graduados_doctorado = graduados.filter(pl.col("NIVEL_CODE") == "DO")

graduados_pregrado = graduados_pregrado.sort("PERIODO",descending=True)

graduados_pregrado = graduados_pregrado.unique(subset=["CODIGO_ESTUDIANTE"],keep="first")

graduados_pregrado= graduados_pregrado.select(['CODIGO_ESTUDIANTE'])

graduados_maestria = graduados_maestria.sort("PERIODO",descending=True)

graduados_maestria = graduados_maestria.unique(subset=["CODIGO_ESTUDIANTE"],keep="first")

graduados_maestria= graduados_maestria.select(['CODIGO_ESTUDIANTE'])

graduados_doctorado = graduados_doctorado.sort("PERIODO",descending=True)

graduados_doctorado = graduados_doctorado.unique(subset=["CODIGO_ESTUDIANTE"],keep="first")

graduados_doctorado= graduados_doctorado.select(['CODIGO_ESTUDIANTE'])

# Crear una nueva columna con un valor fijo
graduados_pregrado = graduados_pregrado.with_columns(
    pl.lit("SI").alias("GRADUADO_PR")  # Crear columna 'GRADUADO_PR' con valor fijo 'SI'
)

graduados_maestria = graduados_maestria.with_columns(
    pl.lit("SI").alias("GRADUADO_MA")  # Crear columna 'GRADUADO_MA' con valor fijo 'SI'
)

graduados_doctorado = graduados_doctorado.with_columns(
    pl.lit("SI").alias("GRADUADO_DO")  # Crear columna 'ESTADO' con valor fijo 'GRADUADO'
)


#MATRICULADO SEMESTRE ACTUAL ? (ULTIMO SEMESTRE DE 10 O 20)

# periodos_a_evaluar_todos_list = (
#     Info_Notas_HA_Actual.lazy()  # Convertir a LazyFrame
#     .with_columns(pl.col("PERIODO").cast(pl.Int32))  # Convertir PERIODO a entero
#     .filter(
#         (pl.col("PERIODO") % 10 == 0) & 
#         ~(pl.col("PERIODO").cast(str).str.ends_with("00"))  # Verificar que no termina en "00"
#     )
#     .select("PERIODO")       # Seleccionar solo la columna PERIODO
#     .unique()                # Obtener valores únicos
#     .sort("PERIODO", descending=True)   # Ordenar en orden descendente
# )

# # Ejecutar la operación Lazy para obtener la lista
# periodos_a_evaluar_todos_list = periodos_a_evaluar_todos_list.collect().to_series().to_list()


periodos_a_evaluar_todos_list = (
    Info_Notas_HA_Actual
    .with_columns(pl.col("PERIODO").cast(pl.Int32))  # Convertir PERIODO a entero
    .filter(
        (pl.col("PERIODO") % 10 == 0) & 
        ~(pl.col("PERIODO").cast(str).str.ends_with("00"))  # Verificar que no termina en "00"
    )
    .select("PERIODO")       # Seleccionar solo la columna PERIODO
    .unique()                # Obtener valores únicos
    .sort("PERIODO",descending=True)   # Ordenar en orden descendente
    .to_series()
    .to_list()               # Convertir a lista
)

ultimo_periodo= periodos_a_evaluar_todos_list[0]
matriculados_ultimo_semestre = Info_Notas_HA_Actual.with_columns(pl.col("PERIODO").cast(pl.Int32)).filter(pl.col("PERIODO") == ultimo_periodo)

# Filtrar por la condición en DESCRIPCION_ESTADO_MATERIA y MATRICULADO
matriculados_ultimo_semestre = matriculados_ultimo_semestre.filter(
    (pl.col("DESCRIPCION_ESTADO_MATERIA") != "RETIRADO - NO PAGO") &
    (pl.col("MATRICULADO") == "SI")
)

# Eliminar duplicados según CODIGO_ESTUDIANTE
matriculados_ultimo_semestre = matriculados_ultimo_semestre.unique(subset=["CODIGO_ESTUDIANTE"])

# Filtrar filas donde CODIGO_ESTUDIANTE no sea nulo
matriculados_ultimo_semestre = matriculados_ultimo_semestre.filter(pl.col("CODIGO_ESTUDIANTE").is_not_null())

matriculados_ultimo_semestre = matriculados_ultimo_semestre.select(["CODIGO_ESTUDIANTE"])

matriculados_ultimo_semestre = matriculados_ultimo_semestre.with_columns(
    pl.lit("SI").alias("MATRICULADO_SEMESTRE_MAS_RECIENTE")  # Crear columna 'MATRICULADO_SEMESTRE_MAS_RECIENTE' con valor fijo 'SI'
)

# CALCULAR EL ESTADO DEL ESTUDIANTE ACTIVO-INACTIVO-DESERTOR

# Función para calcular días a semestres
def obtener_dias_deltatime(days):
    return math.floor(days / 180)

# Función para convertir un periodo en formato YYYY2 o YYYY1 a una fecha
def convert_period_to_date(period):
    year = int(str(period)[:4])
    semester = int(str(period)[-2])
    month = 7 if semester == 2 else 1
    return datetime(year, month, 1)

# Periodo base
periodo_base = ultimo_periodo  # Ejemplo
periodo_actual_date = convert_period_to_date(periodo_base)

data = historia_acadmica

data= data.sort("PERIODO",descending=True)

# Convertir PERIODO a fecha
# Convertir PERIODO a fecha
data = data.with_columns(
    pl.col("PERIODO")
    .cast(pl.Int32)  # Asegurar que PERIODO sea un entero
    .map_elements(convert_period_to_date)  # Aplicar la función personalizada
    .alias("PERIODO_EVALUADO_FECHA")
)

# Calcular SEMESTRES_NO_MATRICULADOS
# Calcular SEMESTRES_NO_MATRICULADOS
data = data.with_columns(
    pl.col("PERIODO_EVALUADO_FECHA")
    .map_elements(lambda x: obtener_dias_deltatime((periodo_actual_date - x).days))  # Aplicar la función personalizada
    .alias("SEMESTRES_NO_MATRICULADOS")
)


# Eliminar duplicados por CODIGO_ESTUDIANTE, conservando el último PERIODO
data = data.sort("PERIODO").unique(subset=["CODIGO_ESTUDIANTE"], keep="first")

# Asignar INACTIVO_DESERTOR_ACTIVO según SEMESTRES_NO_MATRICULADOS
data = data.with_columns(
    pl.when(pl.col("SEMESTRES_NO_MATRICULADOS") <= 1)
    .then(pl.lit("ACTIVO"))
    .when(pl.col("SEMESTRES_NO_MATRICULADOS") == 2)
    .then(pl.lit("INACTIVO"))
    .otherwise(pl.lit("DESERTOR"))
    .alias("INACTIVO_DESERTOR_ACTIVO")
)

# Seleccionar columnas finales
data_no_graduados_general_desert_inactivos = data.select([
    "CODIGO_ESTUDIANTE", 
    "SEMESTRES_NO_MATRICULADOS", 
    "INACTIVO_DESERTOR_ACTIVO"
])

# ULTIMO ESTADO ACADEMICO 

# TOTAL PRUEBAS ACADEMICAS , REINGRESOS, SUSPENSIONES, INCOMPLETOS

# PUNTAJE ADVISE
#Toca sacarlo de un archivo estático



# HA VISTO UNA MATERIA 3 VECES O MÁS?
#Toca calcular la cantidad de materias vista por estudiante





/root/.ipykernel/1274/command-755623736686737-2667203342:114: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  data = data.with_columns(
/root/.ipykernel/1274/command-755623736686737-2667203342:123: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  data = data.with_columns(


In [0]:
#hechos_info_academica

# Paso 3: Ordenar por "PERIODO" en orden descendente
hechos_info_academica = historia_acadmica.sort("PERIODO")

# Paso 4: Seleccionar las columnas necesarias
columnas_seleccionadas = ["PERIODO", "CODIGO_ESTUDIANTE", "CODIGO_ESTADO_ACADEMICO", "DESCRIPCION_ESTADO_ACADEMICO", "SSC",
                          "CREDITO_SEM", "CREDITO_SEM_INTENTADO_INSTITUCIONAL", "CREDITO_SEM_APROBADO_INSTITUCIONAL",
                          "CREDITO_SEM_PGA_INSTITUCIONAL", "PROMEDIO_SEM_INSTITUCIONAL", "CREDITO_SEM_INTENTADO_TRANSFERENCIA",
                          "CREDITO_SEM_APROBADO_TRANSFERENCIA", "CREDITO_SEM_PGA_TRANSFERENCIA", "PROMEDIO_SEM_TRANSFERENCIA",
                          "CREDITO_INTENTADO_GLOBAL", "CREDITO_APROBADO_GLOBAL", "CREDITO_PGA_GLOBAL", "PROMEDIO_GLOBAL",
                          "OPCION_DE_GRADO", "PROMEDIO_ACUM_INSTITUCIONAL", "PROMEDIO_SEM_GLOBAL", "CREDITO_SEM_GLOBAL",
                          "PROMEDIO_ACUM_TRANSFERENCIA", "PUNTOS_CALIDAD_SEM", "PUNTOS_CALIDAD_ACUM", "NUMERO_CRN_RETIRADOS",
                          "TOTAL_CREDIT_RETIRADOS_SEM", "CREDITO_SEM_INTENTADO_TOTAL"]

hechos_info_academica = hechos_info_academica.select(columnas_seleccionadas)

# Paso 5: Realizar join con `info_programas_periodo` (se requiere cargar este DataFrame `info_programas_periodo`)
# Suponiendo que `info_programas_periodo` está en `df_info_programas`

hechos_info_academica = hechos_info_academica.join(info_programas_periodo.select(["PERIODO", "CODIGO_ESTUDIANTE","CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"]), on=["PERIODO", "CODIGO_ESTUDIANTE"], how="left")

# Paso 6: Realizar otro join con `dim_programa`

hechos_info_academica = hechos_info_academica.join(dim_programa.select(["ID_PROGRAMA","CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"]), on=["CODIGO_PROGRAMA_1", "CODIGO_NIVEL_PROGRAMA_1", "CODIGO_PROGRAMA_2", "CODIGO_NIVEL_PROGRAMA_2",
    "CODIGO_PROGRAMA_3", "CODIGO_NIVEL_PROGRAMA_3"], how="left")

# Paso 7: Duplicar las columnas "PERIODO" y "CODIGO_ESTUDIANTE"
hechos_info_academica = hechos_info_academica.with_columns(
    pl.col("PERIODO").alias("PERIODO_COPIA"),
    pl.col("CODIGO_ESTUDIANTE").alias("CODIGO_ESTUDIANTE_COPIA")
)

# Paso 8: Crear columna combinada "PERIODO_CODIGO_ESTUDIANTE"
hechos_info_academica = hechos_info_academica.with_columns(
    (pl.col("PERIODO_COPIA") + pl.col("CODIGO_ESTUDIANTE_COPIA")).alias("PERIODO_CODIGO")
)

# Paso 9: Join con `generacion_ids_grupo` (requiere cargar `generacion_ids_grupo`)
hechos_info_academica = hechos_info_academica.join(generacion_ids_grupo.select(["PERIODO_CODIGO", "ID_GRUPO"]), on="PERIODO_CODIGO", how="left")

# Paso 10: Reemplazar valores nulos con "No especificado"
hechos_info_academica = hechos_info_academica.fill_null("No especificado")

# Paso 11: Ordenar por "SSC" en orden descendente y eliminar duplicados en "PERIODO" y "CODIGO_ESTUDIANTE"
hechos_info_academica = hechos_info_academica.sort("SSC", descending=True).unique(subset=["PERIODO", "CODIGO_ESTUDIANTE"],keep="first")

# Paso 12: Join con `agrupacion_creditos` y expandir las columnas
# Suponiendo que `agrupacion_creditos` ya está cargado
hechos_info_academica= hechos_info_academica.join(data_no_graduados_general_desert_inactivos, on=["CODIGO_ESTUDIANTE"], how="left")

hechos_info_academica = hechos_info_academica.join(matriculados_ultimo_semestre, on=["CODIGO_ESTUDIANTE"], how="left")

hechos_info_academica = hechos_info_academica.join(graduados_doctorado, on=["CODIGO_ESTUDIANTE"], how="left")

hechos_info_academica = hechos_info_academica.join(graduados_maestria, on=["CODIGO_ESTUDIANTE"], how="left")

hechos_info_academica = hechos_info_academica.join(graduados_pregrado, on=["CODIGO_ESTUDIANTE"], how="left")

hechos_info_academica = hechos_info_academica.join(agrupacion_creditos, on=["PERIODO", "CODIGO_ESTUDIANTE"], how="left")

hechos_info_academica = hechos_info_academica.with_columns(pl.col("TOTAL_CREDITOS").cast(int))

# Aplicar condiciones para crear la columna "TOTAL_SEMESTRES_MATRICULADOS"
hechos_info_academica = hechos_info_academica.with_columns([
    pl.when(
        (pl.col("CODIGO_NIVEL_PROGRAMA_1") == "PR") & (pl.col("CODIGO_PROGRAMA_1") == "MEDI") & (pl.col("TOTAL_CREDITOS") > 12)
    ).then(1)
    .when(
        (pl.col("CODIGO_NIVEL_PROGRAMA_1") == "PR") & (pl.col("CODIGO_PROGRAMA_1") == "MEDI") & (pl.col("TOTAL_CREDITOS") > 7) & (pl.col("TOTAL_CREDITOS") <= 12)
    ).then(0.5)
    .when(
        (pl.col("CODIGO_NIVEL_PROGRAMA_1") == "PR") & (pl.col("CODIGO_PROGRAMA_1") != "MEDI") & (pl.col("TOTAL_CREDITOS") > 9)
    ).then(1)
    .when(
        (pl.col("CODIGO_NIVEL_PROGRAMA_1") == "PR") & (pl.col("CODIGO_PROGRAMA_1") != "MEDI") & (pl.col("TOTAL_CREDITOS") > 4) & (pl.col("TOTAL_CREDITOS") <= 9)
    ).then(0.5)
    .when(
        (pl.col("CODIGO_NIVEL_PROGRAMA_1") == "MA") & (pl.col("TOTAL_CREDITOS") >= 8)
    ).then(1)
    .when(
        (pl.col("CODIGO_NIVEL_PROGRAMA_1") == "MA") & (pl.col("TOTAL_CREDITOS") <= 4)
    ).then(0.5)
    .otherwise(0.25)
    .alias("TOTAL_SEMESTRES_MATRICULADOS")
])
# Resultado final

In [0]:
# hechos_info_academica.filter(pl.col("CODIGO_ESTUDIANTE")=="201913911")


# Limpieza y preparación para exportación a postgresql


In [0]:
dim_curso.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_curso.parquet')
dim_grupo.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_grupo.parquet')
dim_programa.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_programa.parquet')
dim_tipo_financiamiento.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_tipo_financiamiento.parquet')
dim_union_grupo_tipo.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_union_grupo_tipo.parquet')
dim_estudiantes.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_estudiantes.parquet')
dim_estudiantes_magister.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_estudiantes_magister.parquet')
dim_profes.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_profesor.parquet')
dim_periodo.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_periodo.parquet')

dim_horario.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_horario.parquet')
dim_union_grupo_horario.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_union_grupo_horario.parquet')
dim_grupo_horario.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/dim_grupo_horario.parquet')

hechos_estudiante_curso.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/hechos_estudiante_curso.parquet')

hechos_info_academica.write_parquet('/dbfs/mnt/productos-datos-observatorio/modelo_multidimensional/hechos_info_academica.parquet')




In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def hechos_info_academica_func():
  df_pandas = hechos_info_academica.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def hechos_estudiante_curso_func():

  df_pandas = hechos_estudiante_curso.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_curso_func():
  df_pandas = dim_curso.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_grupo_func():
  df_pandas = dim_grupo.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_programa_func():
  df_pandas = dim_programa.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_tipo_financiamiento_func():
  df_pandas = dim_tipo_financiamiento.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_union_grupo_tipo_func():
  df_pandas = dim_union_grupo_tipo.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_estudiantes_func():
  df_pandas = dim_estudiantes.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_estudiantes_magister_func():
  df_pandas = dim_estudiantes_magister.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
# @dlt.table(
#   comment="The raw sales orders, ingested from /databricks-datasets.",
#   table_properties={
#     "myCompanyPipeline.quality": "bronze",
#     "pipelines.autoOptimize.managed": "true"
#   }
# )

# def dim_union_grupo_tipo_func():
#   df_pandas = dim_union_grupo_tipo.to_pandas()
#   # Crear una sesión de Spark
#   spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

#   # Convertir el DataFrame de Pandas a un DataFrame de Spark
#   df_spark = spark.createDataFrame(df_pandas)
#   return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_profes_func():
  df_pandas = dim_profes.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_horario_func():
  df_pandas = dim_horario.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
#dim_union_grupo_horario
#dim_grupo_horario

@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_union_grupo_horario_func():
  df_pandas = dim_union_grupo_horario.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_grupo_horario_func():
  df_pandas = dim_grupo_horario.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data

In [0]:
@dlt.table(
  comment="The raw sales orders, ingested from /databricks-datasets.",
  table_properties={
    "myCompanyPipeline.quality": "bronze",
    "pipelines.autoOptimize.managed": "true"
  }
)

def dim_periodo_func():
  df_pandas = dim_periodo.to_pandas()
  # Crear una sesión de Spark
  spark = SparkSession.builder.appName("PolarsToSpark").getOrCreate()

  # Convertir el DataFrame de Pandas a un DataFrame de Spark
  df_spark = spark.createDataFrame(df_pandas)
  return df_spark

com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.data